ESTADÍSTICAS CODECARBON EN MI ORDENADOR

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy import stats

# Configuración de estilo para gráficos más atractivos
plt.style.use('seaborn-v0_8-whitegrid')     # Estilo base de seaborn
plt.rcParams['figure.figsize'] = (12, 7)    # Tamaño más grande
plt.rcParams['font.size'] = 12              # Texto más grande
plt.rcParams['axes.labelsize'] = 14         # Etiquetas más grandes
plt.rcParams['axes.titlesize'] = 16         # Título más grande
plt.rcParams['xtick.labelsize'] = 12        # Etiquetas del eje X más grandes
plt.rcParams['ytick.labelsize'] = 12        # Etiquetas del eje Y más grandes
plt.rcParams['legend.fontsize'] = 12        # Leyenda más grande
plt.rcParams['axes.titlepad'] = 20          # Espacio para el título
plt.rcParams['axes.spines.top'] = False     # Eliminar línea superior
plt.rcParams['axes.spines.right'] = False   # Eliminar línea derecha

# Rutas de los 4 ficheros CodeCarbon del ordenador
files_ordenador = {
    "ECJ": "codecarbon_ECJ_ordenador.xlsx",
    "Inspyred": "codecarbon_inspyred_ordenador_3.xlsx",
    "DEAP": "codecarbon_DEAP_ordenador.xlsx",
    "Paradiseo": "codecarbon_paradiseo_ordenador.xlsx"
}

# Columnas a estudiar
cols_interes = [
    "duration", 
    "emissions", 
    "emissions_rate",
    "energy_consumed", 
    "cpu_energy", 
    "ram_energy", 
    "cpu_power", 
    "ram_power"
]

# Mapa manual de configuraciones (fila → descripción)
config_map = {
    0: "2^6, 0.01",
    1: "2^6, 0.2",
    2: "2^6, 0.8",
    3: "2^10, 0.01",
    4: "2^10, 0.2",
    5: "2^10, 0.8",
    6: "2^14, 0.01",
    7: "2^14, 0.2",
    8: "2^14, 0.8",
}

# Orden correcto de configuraciones para usar en gráficos
config_order = [config_map[i] for i in range(9)]

# Colores personalizados para cada framework
framework_colors = {
    "ECJ": "#1f77b4",       # Azul
    "Inspyred": "#ff7f0e",  # Naranja
    "DEAP": "#2ca02c",      # Verde
    "Paradiseo": "#d62728"  # Rojo
}

df_list = []
for fw_name, fpath in files_ordenador.items():
    # Leemos la hoja "MEDIAS", donde están las 9 configuraciones
    temp = pd.read_excel(fpath, sheet_name="MEDIAS")
    
    # Seleccionamos solo las columnas que existan
    existing_cols = [c for c in cols_interes if c in temp.columns]
    df_temp = temp[existing_cols].copy()
    
    # Añadimos la columna 'framework'
    df_temp["framework"] = fw_name
    
    # Reseteamos el index para poder mapear 0..8
    df_temp.reset_index(drop=True, inplace=True)
    # Asignamos la etiqueta de configuración
    df_temp["config"] = df_temp.index.map(config_map)
    
    df_list.append(df_temp)

# Concatenamos en un solo DF
df_all = pd.concat(df_list, ignore_index=True).fillna(0)

def plot_grouped_bars(data, x_col, group_col, y_col, title, ylabel, y_min_pct=0.9, log_scale=False):
    """
    Crea un gráfico de barras agrupadas de la columna 'y_col',
    con 'x_col' en el eje X y cada valor de 'group_col' como una barra.
    Destaca la barra con el mayor valor en cada categoría (fila).
    
    Args:
        data: DataFrame con los datos
        x_col: Columna para el eje X (categorías)
        group_col: Columna para agrupar (diferentes barras)
        y_col: Columna con los valores a mostrar
        title: Título del gráfico
        ylabel: Etiqueta del eje Y
        y_min_pct: Porcentaje del valor mínimo para ajustar el eje Y (0-1)
        log_scale: Si es True, usa escala logarítmica en el eje Y
    """
    # 1) Pasamos a formato ancho: filas = config, columnas = framework, valores = y_col
    pivoted = data.pivot(index=x_col, columns=group_col, values=y_col)
    
    # Reordenamos las filas según el orden definido en config_order
    pivoted = pivoted.reindex(config_order)
    
    # Obtenemos los valores mínimos y máximos para ajustar el eje Y
    y_min = pivoted.values.min() * y_min_pct
    y_max = pivoted.values.max() * 1.15  # 15% de margen superior
    
    # Creamos el gráfico con más espacio a la derecha para la leyenda
    fig, ax = plt.subplots(figsize=(14, 7))  # Figura más ancha para acomodar la leyenda
    pivoted.plot(
        kind="bar", 
        ax=ax,
        color=[framework_colors.get(fw, "#333333") for fw in pivoted.columns],
        width=0.7
    )
    
    # Configuración de escala y límites
    if log_scale:
        plt.yscale('log')  # Escala logarítmica para el eje Y
        # Ajustamos el límite superior para evitar que las etiquetas se salgan
        plt.ylim(top=pivoted.values.max() * 1.3)  # 30% de margen superior
    else:
        plt.ylim(y_min, y_max)
    
    # 4) Título y etiquetas con estilo mejorado
    plt.title(title, fontweight='bold', pad=20)
    plt.xlabel("Configuration", fontweight='bold')
    plt.ylabel(ylabel, fontweight='bold')
    plt.xticks(rotation=30, ha="right")
    
    # 5) Destacamos la barra mayor en cada fila (configuración)
    for row_idx, config_name in enumerate(pivoted.index):
        # Obtenemos la fila con sus 4 frameworks y valores
        row_values = pivoted.loc[config_name]
        # Nombre de la columna (framework) con el valor máximo en esta fila
        max_col = row_values.idxmax()
        # Valor máximo
        max_value = row_values[max_col]
        # Índice (0..3) que corresponde a ese framework en pivoted.columns
        col_idx = list(pivoted.columns).index(max_col)
        
        # Localizamos la barra que corresponde al row_idx en ese contenedor
        bar = ax.containers[col_idx][row_idx]
        
        # Añadimos etiqueta solo a la barra con el valor máximo
        # Ajustamos la posición de la etiqueta para escala logarítmica
        if log_scale:
            label_y_pos = max_value * 1.05
        else:
            label_y_pos = max_value * 1.01
              
        ax.text(
            bar.get_x() + bar.get_width()/2,
            label_y_pos,
            f"{max_value:.4g}",
            ha='center',
            va='bottom',
            fontweight='bold',
            color='black',
            fontsize=11
        )
    
    # 6) Mejoramos la leyenda y la colocamos a la derecha del gráfico
    legend = plt.legend(title="Framework", frameon=True, loc='center left', bbox_to_anchor=(1.0, 0.5))
    legend.get_frame().set_facecolor('#F8F8F8')
    legend.get_frame().set_edgecolor('#CCCCCC')
    
    # 7) Añadimos grid para facilitar la lectura
    plt.grid(axis='both', linestyle='--', alpha=0.7)
    
    # Ajustamos el layout teniendo en cuenta el espacio para la leyenda
    plt.tight_layout(rect=[0, 0, 0.85, 1])
    
    plt.show()

# Función mejorada para mostrar líneas con tendencias adaptativas
def plot_trends(data, y_col, title, ylabel, log_scale=False, show_labels=True):
    """
    Crea un gráfico de líneas con líneas de tendencia adaptativas que se ajustan mejor
    a datos con cambios bruscos.
    
    Args:
        data: DataFrame con los datos
        y_col: Columna con los valores a mostrar
        title: Título del gráfico
        ylabel: Etiqueta del eje Y
        log_scale: Si es True, usa escala logarítmica en el eje Y
        show_labels: Si es True, muestra las etiquetas numéricas en los puntos
    """
    # Creamos una figura
    fig, ax = plt.subplots(figsize=(14, 7))
    
    # Extraemos los frameworks únicos
    frameworks = data['framework'].unique()
    
    # Extraemos los índices de configuración para representar mejor la tendencia
    config_indices = list(range(len(config_order)))
    
    # Almacenar los valores R² para incluirlos en la leyenda
    r_squared_values = {}
    
    # Para cada framework, graficamos una línea
    for fw in frameworks:
        fw_data = data[data['framework'] == fw].copy()
        
        # Preparamos arrays para valores e índices
        fw_values = []
        fw_indices = []
        
        # Para cada configuración, extraemos el valor si existe
        for i, config in enumerate(config_order):
            config_data = fw_data[fw_data['config'] == config]
            if len(config_data) > 0:
                val = config_data[y_col].values[0]
                # Para escala logarítmica, aseguramos que los valores sean positivos
                if log_scale and (val <= 0 or np.isnan(val)):
                    continue
                fw_values.append(val)
                fw_indices.append(i)
                
        # Si no hay suficientes datos, continuamos con el siguiente framework
        if len(fw_indices) < 3:
            continue
        
        # Dibujamos la línea principal con todos los puntos
        line, = ax.plot(fw_indices, fw_values, 'o-', 
                      label=fw,
                      color=framework_colors.get(fw),
                      linewidth=2, markersize=8)
        
        # Añadimos etiquetas si está habilitado
        if show_labels:
            for i, val in zip(fw_indices, fw_values):
                if not np.isnan(val) and (val > 0 or not log_scale):
                    ax.text(i, val * (1.05 if log_scale else 1.01), 
                           f"{val:.4g}", ha='center', va='bottom',
                           color=framework_colors.get(fw),
                           fontsize=9, fontweight='bold')
        
        # Generamos más puntos para suavizar la línea de tendencia
        if len(fw_indices) >= 3:
            # Usamos un ajuste polinómico (grado 2) para capturar cambios no lineales
            # El grado 2 permite capturar una curvatura básica, suficiente para estos datos
            z = np.polyfit(fw_indices, fw_values, 2)
            p = np.poly1d(z)
            
            # Generamos puntos adicionales para suavizar la curva
            x_smooth = np.linspace(min(fw_indices), max(fw_indices), 100)
            y_smooth = p(x_smooth)
            
            # Dibujamos la línea de tendencia suavizada
            ax.plot(x_smooth, y_smooth, '--', 
                   color=framework_colors.get(fw), 
                   linewidth=1.5, alpha=0.7)
            
            # Calculamos R² para evaluar la calidad del ajuste
            y_pred = p(fw_indices)
            ss_total = np.sum((fw_values - np.mean(fw_values))**2)
            ss_residual = np.sum((fw_values - y_pred)**2)
            r_squared = 1 - (ss_residual / ss_total)
            r_squared_values[fw] = r_squared
        else:
            r_squared_values[fw] = float('nan')
    
    # Configuración de escala y límites
    if log_scale:
        ax.set_yscale('log')
    
    # Título y etiquetas
    ax.set_title(f"{title} with Trend Lines", fontweight='bold', pad=20)
    ax.set_xticks(config_indices)
    ax.set_xticklabels(config_order, rotation=30, ha='right')
    ax.set_xlabel("Configuration", fontweight='bold')
    ax.set_ylabel(ylabel, fontweight='bold')
    
    # Añadir áreas sombreadas para separar tamaños de población
    pop_sizes = ["2^6", "2^10", "2^14"]
    colors = ["#f0f0f0", "#e8e8e8", "#f0f0f0"]
    
    for p_idx, pop in enumerate(pop_sizes):
        pop_configs = [i for i, c in enumerate(config_order) if c.startswith(pop)]
        if pop_configs:
            start, end = min(pop_configs), max(pop_configs)
            ax.axvspan(start - 0.5, end + 0.5, alpha=0.2, color=colors[p_idx])
            ax.text((start + end) / 2, 0.98, 
                   f"Population {pop}", ha='center', fontsize=10, alpha=0.7,
                   transform=ax.get_xaxis_transform())
    
    # Crear etiquetas personalizadas para la leyenda que incluyan R²
    custom_labels = []
    custom_handles = []
    
    # Obtenemos los handles actuales y sus etiquetas
    handles, labels = ax.get_legend_handles_labels()
    
    for handle, label in zip(handles, labels):
        if label in r_squared_values:
            r2 = r_squared_values[label]
            if not np.isnan(r2):
                # Incluimos el valor R² en la etiqueta
                custom_labels.append(f"{label} (R²={r2:.2f})")
            else:
                custom_labels.append(label)
            custom_handles.append(handle)
    
    # Crear la leyenda con las etiquetas personalizadas
    legend = ax.legend(custom_handles, custom_labels, title="Framework", 
                      frameon=True, loc='center left', bbox_to_anchor=(1.0, 0.5))
    legend.get_frame().set_facecolor('#F8F8F8')
    legend.get_frame().set_edgecolor('#CCCCCC')
    
    # Grid y layout
    ax.grid(axis='both', linestyle='--', alpha=0.7)
    plt.tight_layout(rect=[0, 0, 0.85, 1])
    
    plt.show()
# ─────────────────────────────────────────────────────────
# Generamos gráficos de barras y de tendencias para cada columna
# ─────────────────────────────────────────────────────────
if "emissions" in df_all.columns:
    # Gráfico de barras original
    plot_grouped_bars(
        data=df_all,
        x_col="config",
        group_col="framework",
        y_col="emissions",
        title="CO₂ Emissions (Laptop)",
        ylabel="kg CO₂",
        y_min_pct=0.85  # Empieza en el 85% del valor mínimo
    )
    
    # Nuevo gráfico con líneas de tendencia
    plot_trends(
        data=df_all,
        y_col="emissions",
        title="CO₂ Emissions (Laptop)",
        ylabel="kg CO₂",
        log_scale=False,
        show_labels=False  # Cambiar a False para quitar etiquetas numéricas
    )

if "emissions_rate" in df_all.columns:
    # Gráfico de barras original
    plot_grouped_bars(
        data=df_all,
        x_col="config",
        group_col="framework",
        y_col="emissions_rate",
        title="Emissions Rate (Laptop)",
        ylabel="kg CO₂/h",
        y_min_pct=0.85
    )
    
    # Nuevo gráfico con líneas de tendencia
    plot_trends(
        data=df_all,
        y_col="emissions_rate",
        title="Emissions Rate (Laptop)",
        ylabel="kg CO₂/h",
        log_scale=False,
        show_labels=False
    )

if "energy_consumed" in df_all.columns:
    # Gráfico de barras original
    plot_grouped_bars(
        data=df_all,
        x_col="config",
        group_col="framework",
        y_col="energy_consumed",
        title="Energy consumed (Laptop)",
        ylabel="kWh",
        y_min_pct=0.85
    )
    
    # Nuevo gráfico con líneas de tendencia
    plot_trends(
        data=df_all,
        y_col="energy_consumed",
        title="Energy consumed (Laptop)",
        ylabel="kWh",
        log_scale=False,
        show_labels=False
    )

if "cpu_energy" in df_all.columns:
    # Gráfico de barras original
    plot_grouped_bars(
        data=df_all,
        x_col="config",
        group_col="framework",
        y_col="cpu_energy",
        title="CPU energy (Laptop)",
        ylabel="kWh",
        y_min_pct=0.85
    )
    
    # Nuevo gráfico con líneas de tendencia
    plot_trends(
        data=df_all,
        y_col="cpu_energy",
        title="CPU energy (Laptop)",
        ylabel="kWh",
        log_scale=False,
        show_labels=False
    )

if "ram_energy" in df_all.columns:
    # Gráfico de barras original
    plot_grouped_bars(
        data=df_all,
        x_col="config",
        group_col="framework",
        y_col="ram_energy",
        title="RAM energy (Laptop)",
        ylabel="kWh",
        y_min_pct=0.85,
        log_scale=True
    )
    
    # Nuevo gráfico con líneas de tendencia
    plot_trends(
        data=df_all,
        y_col="ram_energy",
        title="RAM energy (Laptop)",
        ylabel="kWh",
        log_scale=True,
        show_labels=False
    )

if "cpu_power" in df_all.columns:
    # Gráfico de barras original
    plot_grouped_bars(
        data=df_all,
        x_col="config",
        group_col="framework",
        y_col="cpu_power",
        title="CPU power (Laptop)",
        ylabel="W",
        y_min_pct=0.85,
        log_scale=True
    )
    
    # Nuevo gráfico con líneas de tendencia
    plot_trends(
        data=df_all,
        y_col="cpu_power",
        title="CPU power (Laptop)",
        ylabel="W",
        log_scale=True,
        show_labels=False
    )

if "ram_power" in df_all.columns:
    # Gráfico de barras original
    plot_grouped_bars(
        data=df_all,
        x_col="config",
        group_col="framework",
        y_col="ram_power",
        title="RAM power (Laptop)",
        ylabel="W",
        y_min_pct=0.85,
        log_scale=True
    )
    
    # Nuevo gráfico con líneas de tendencia
    plot_trends(
        data=df_all,
        y_col="ram_power",
        title="RAM power (Laptop)",
        ylabel="W",
        log_scale=True,
        show_labels=False
    )

if "duration" in df_all.columns:
    # Gráfico de barras original
    plot_grouped_bars(
        data=df_all,
        x_col="config",
        group_col="framework",
        y_col="duration",
        title="Running time (Laptop)",
        ylabel="seconds",
        y_min_pct=0.85
    )
    
    # Nuevo gráfico con líneas de tendencia
    plot_trends(
        data=df_all,
        y_col="duration",
        title="Running time (Laptop)",
        ylabel="seconds",
        log_scale=False,
        show_labels=False
    )

ESTADÍSTICAS CODECARBON EN EL SERVIDOR

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy import stats

# Configuración de estilo para gráficos más atractivos
plt.style.use('seaborn-v0_8-whitegrid')     # Estilo base de seaborn
plt.rcParams['figure.figsize'] = (12, 7)    # Tamaño más grande
plt.rcParams['font.size'] = 12              # Texto más grande
plt.rcParams['axes.labelsize'] = 14         # Etiquetas más grandes
plt.rcParams['axes.titlesize'] = 16         # Título más grande
plt.rcParams['xtick.labelsize'] = 12        # Etiquetas del eje X más grandes
plt.rcParams['ytick.labelsize'] = 12        # Etiquetas del eje Y más grandes
plt.rcParams['legend.fontsize'] = 12        # Leyenda más grande
plt.rcParams['axes.titlepad'] = 20          # Espacio para el título
plt.rcParams['axes.spines.top'] = False     # Eliminar línea superior
plt.rcParams['axes.spines.right'] = False   # Eliminar línea derecha

# Rutas de los 4 ficheros CodeCarbon del servidor
files_pablo = {
    "ECJ": "codecarbon_ECJ_pablo.xlsx",
    "Inspyred": "codecarbon_inspyred_pablo_3.xlsx",
    "DEAP": "codecarbon_DEAP_pablo.xlsx",
    "Paradiseo": "codecarbon_paradiseo_pablo.xlsx"
}

# Columnas para estudiar
cols_interes = [
    "duration", 
    "emissions", 
    "emissions_rate",
    "energy_consumed", 
    "cpu_energy", 
    "ram_energy", 
    "cpu_power", 
    "ram_power"
]

# Mapa manual de configuraciones (fila → descripción)
config_map = {
    0: "2^6, 0.01",
    1: "2^6, 0.2",
    2: "2^6, 0.8",
    3: "2^10, 0.01",
    4: "2^10, 0.2",
    5: "2^10, 0.8",
    6: "2^14, 0.01",
    7: "2^14, 0.2",
    8: "2^14, 0.8",
}

# Orden correcto de configuraciones para usar en gráficos
config_order = [config_map[i] for i in range(9)]

# Colores personalizados para cada framework
framework_colors = {
    "ECJ": "#1f77b4",       # Azul
    "Inspyred": "#ff7f0e",  # Naranja
    "DEAP": "#2ca02c",      # Verde
    "Paradiseo": "#d62728"  # Rojo
}

df_list = []
for fw_name, fpath in files_pablo.items():
    # Leemos la hoja "MEDIAS", donde están las 9 configuraciones
    temp = pd.read_excel(fpath, sheet_name="MEDIAS")
    
    # Seleccionamos solo las columnas que existan
    existing_cols = [c for c in cols_interes if c in temp.columns]
    df_temp = temp[existing_cols].copy()
    
    # Añadimos la columna 'framework'
    df_temp["framework"] = fw_name
    
    # Reseteamos el index para poder mapear 0..8
    df_temp.reset_index(drop=True, inplace=True)
    # Asignamos la etiqueta de configuración
    df_temp["config"] = df_temp.index.map(config_map)
    
    df_list.append(df_temp)

# Concatenamos en un solo DF
df_all = pd.concat(df_list, ignore_index=True).fillna(0)

def plot_grouped_bars(data, x_col, group_col, y_col, title, ylabel, y_min_pct=0.9, log_scale=False):
    """
    Crea un gráfico de barras agrupadas de la columna 'y_col',
    con 'x_col' en el eje X y cada valor de 'group_col' como una barra.
    Destaca la barra con el mayor valor en cada categoría (fila).
    
    Args:
        data: DataFrame con los datos
        x_col: Columna para el eje X (categorías)
        group_col: Columna para agrupar (diferentes barras)
        y_col: Columna con los valores a mostrar
        title: Título del gráfico
        ylabel: Etiqueta del eje Y
        y_min_pct: Porcentaje del valor mínimo para ajustar el eje Y (0-1)
        log_scale: Si es True, usa escala logarítmica en el eje Y
    """
    # 1) Pasamos a formato ancho: filas = config, columnas = framework, valores = y_col
    pivoted = data.pivot(index=x_col, columns=group_col, values=y_col)
    
    # Reordenamos las filas según el orden definido en config_order
    pivoted = pivoted.reindex(config_order)
    
    # Obtenemos los valores mínimos y máximos para ajustar el eje Y
    y_min = pivoted.values.min() * y_min_pct
    y_max = pivoted.values.max() * 1.15  # 15% de margen superior
    
    # Creamos el gráfico con más espacio a la derecha para la leyenda
    fig, ax = plt.subplots(figsize=(14, 7))  # Figura más ancha para acomodar la leyenda
    pivoted.plot(
        kind="bar", 
        ax=ax,
        color=[framework_colors.get(fw, "#333333") for fw in pivoted.columns],
        width=0.7
    )
    
    # Configuración de escala y límites
    if log_scale:
        plt.yscale('log')  # Escala logarítmica para el eje Y
        # Ajustamos el límite superior para evitar que las etiquetas se salgan
        plt.ylim(top=pivoted.values.max() * 1.3)  # 30% de margen superior
    else:
        plt.ylim(y_min, y_max)
    
    # 4) Título y etiquetas con estilo mejorado
    plt.title(title, fontweight='bold', pad=20)
    plt.xlabel("Configuration", fontweight='bold')
    plt.ylabel(ylabel, fontweight='bold')
    plt.xticks(rotation=30, ha="right")
    
    # 5) Destacamos la barra mayor en cada fila (configuración)
    for row_idx, config_name in enumerate(pivoted.index):
        # Obtenemos la fila con sus 4 frameworks y valores
        row_values = pivoted.loc[config_name]
        # Nombre de la columna (framework) con el valor máximo en esta fila
        max_col = row_values.idxmax()
        # Valor máximo
        max_value = row_values[max_col]
        # Índice (0..3) que corresponde a ese framework en pivoted.columns
        col_idx = list(pivoted.columns).index(max_col)
        
        # Localizamos la barra que corresponde al row_idx en ese contenedor
        bar = ax.containers[col_idx][row_idx]
        
        # Añadimos etiqueta solo a la barra con el valor máximo
        # Ajustamos la posición de la etiqueta para escala logarítmica
        if log_scale:
            label_y_pos = max_value * 1.05
        else:
            label_y_pos = max_value * 1.01
        
        ax.text(
            bar.get_x() + bar.get_width()/2,
            label_y_pos,
            f"{max_value:.4g}",
            ha='center',
            va='bottom',
            fontweight='bold',
            color='black',
            fontsize=11
        )
    
    # 6) Mejoramos la leyenda y la colocamos a la derecha del gráfico
    legend = plt.legend(title="Framework", frameon=True, loc='center left', bbox_to_anchor=(1.0, 0.5))
    legend.get_frame().set_facecolor('#F8F8F8')
    legend.get_frame().set_edgecolor('#CCCCCC')
    
    # 7) Añadimos grid para facilitar la lectura
    plt.grid(axis='both', linestyle='--', alpha=0.7)
    
    # Ajustamos el layout teniendo en cuenta el espacio para la leyenda
    plt.tight_layout(rect=[0, 0, 0.85, 1])
    
    plt.show()

# Función mejorada para mostrar líneas con tendencias adaptativas
def plot_trends(data, y_col, title, ylabel, log_scale=False, show_labels=True):
    """
    Crea un gráfico de líneas con líneas de tendencia adaptativas que se ajustan mejor
    a datos con cambios bruscos.
    
    Args:
        data: DataFrame con los datos
        y_col: Columna con los valores a mostrar
        title: Título del gráfico
        ylabel: Etiqueta del eje Y
        log_scale: Si es True, usa escala logarítmica en el eje Y
        show_labels: Si es True, muestra las etiquetas numéricas en los puntos
    """
    # Creamos una figura
    fig, ax = plt.subplots(figsize=(14, 7))
    
    # Extraemos los frameworks únicos
    frameworks = data['framework'].unique()
    
    # Extraemos los índices de configuración para representar mejor la tendencia
    config_indices = list(range(len(config_order)))
    
    # Almacenar los valores R² para incluirlos en la leyenda
    r_squared_values = {}
    
    # Para cada framework, graficamos una línea
    for fw in frameworks:
        fw_data = data[data['framework'] == fw].copy()
        
        # Preparamos arrays para valores e índices
        fw_values = []
        fw_indices = []
        
        # Para cada configuración, extraemos el valor si existe
        for i, config in enumerate(config_order):
            config_data = fw_data[fw_data['config'] == config]
            if len(config_data) > 0:
                val = config_data[y_col].values[0]
                # Para escala logarítmica, aseguramos que los valores sean positivos
                if log_scale and (val <= 0 or np.isnan(val)):
                    continue
                fw_values.append(val)
                fw_indices.append(i)
                
        # Si no hay suficientes datos, continuamos con el siguiente framework
        if len(fw_indices) < 3:
            continue
        
        # Dibujamos la línea principal con todos los puntos
        line, = ax.plot(fw_indices, fw_values, 'o-', 
                      label=fw,
                      color=framework_colors.get(fw),
                      linewidth=2, markersize=8)
        
        # Añadimos etiquetas si está habilitado
        if show_labels:
            for i, val in zip(fw_indices, fw_values):
                if not np.isnan(val) and (val > 0 or not log_scale):
                    ax.text(i, val * (1.05 if log_scale else 1.01), 
                           f"{val:.4g}", ha='center', va='bottom',
                           color=framework_colors.get(fw),
                           fontsize=9, fontweight='bold')
        
        # Generamos más puntos para suavizar la línea de tendencia
        if len(fw_indices) >= 3:
            # Usamos un ajuste polinómico (grado 2) para capturar cambios no lineales
            # El grado 2 permite capturar una curvatura básica, suficiente para estos datos
            z = np.polyfit(fw_indices, fw_values, 2)
            p = np.poly1d(z)
            
            # Generamos puntos adicionales para suavizar la curva
            x_smooth = np.linspace(min(fw_indices), max(fw_indices), 100)
            y_smooth = p(x_smooth)
            
            # Dibujamos la línea de tendencia suavizada
            ax.plot(x_smooth, y_smooth, '--', 
                   color=framework_colors.get(fw), 
                   linewidth=1.5, alpha=0.7)
            
            # Calculamos R² para evaluar la calidad del ajuste
            y_pred = p(fw_indices)
            ss_total = np.sum((fw_values - np.mean(fw_values))**2)
            ss_residual = np.sum((fw_values - y_pred)**2)
            r_squared = 1 - (ss_residual / ss_total)
            r_squared_values[fw] = r_squared
        else:
            r_squared_values[fw] = float('nan')
    
    # Configuración de escala y límites
    if log_scale:
        ax.set_yscale('log')
    
    # Título y etiquetas
    ax.set_title(f"{title} with Trend Lines", fontweight='bold', pad=20)
    ax.set_xticks(config_indices)
    ax.set_xticklabels(config_order, rotation=30, ha='right')
    ax.set_xlabel("Configuration", fontweight='bold')
    ax.set_ylabel(ylabel, fontweight='bold')
    
    # Añadir áreas sombreadas para separar tamaños de población
    pop_sizes = ["2^6", "2^10", "2^14"]
    colors = ["#f0f0f0", "#e8e8e8", "#f0f0f0"]
    
    for p_idx, pop in enumerate(pop_sizes):
        pop_configs = [i for i, c in enumerate(config_order) if c.startswith(pop)]
        if pop_configs:
            start, end = min(pop_configs), max(pop_configs)
            ax.axvspan(start - 0.5, end + 0.5, alpha=0.2, color=colors[p_idx])
            ax.text((start + end) / 2, 0.98, 
                   f"Population {pop}", ha='center', fontsize=10, alpha=0.7,
                   transform=ax.get_xaxis_transform())
    
    # Crear etiquetas personalizadas para la leyenda que incluyan R²
    custom_labels = []
    custom_handles = []
    
    # Obtenemos los handles actuales y sus etiquetas
    handles, labels = ax.get_legend_handles_labels()
    
    for handle, label in zip(handles, labels):
        if label in r_squared_values:
            r2 = r_squared_values[label]
            if not np.isnan(r2):
                # Incluimos el valor R² en la etiqueta
                custom_labels.append(f"{label} (R²={r2:.2f})")
            else:
                custom_labels.append(label)
            custom_handles.append(handle)
    
    # Crear la leyenda con las etiquetas personalizadas
    legend = ax.legend(custom_handles, custom_labels, title="Framework", 
                      frameon=True, loc='center left', bbox_to_anchor=(1.0, 0.5))
    legend.get_frame().set_facecolor('#F8F8F8')
    legend.get_frame().set_edgecolor('#CCCCCC')
    
    # Grid y layout
    ax.grid(axis='both', linestyle='--', alpha=0.7)
    plt.tight_layout(rect=[0, 0, 0.85, 1])
    
    plt.show()
# ─────────────────────────────────────────────────────────
# Generamos gráficos de barras y de tendencias para cada columna
# ─────────────────────────────────────────────────────────
if "emissions" in df_all.columns:
    # Gráfico de barras original
    plot_grouped_bars(
        data=df_all,
        x_col="config",
        group_col="framework",
        y_col="emissions",
        title="CO₂ Emissions (Server)",
        ylabel="kg CO₂",
        y_min_pct=0.85  # Empieza en el 85% del valor mínimo
    )
    
    # Nuevo gráfico con líneas de tendencia
    plot_trends(
        data=df_all,
        y_col="emissions",
        title="CO₂ Emissions (Server)",
        ylabel="kg CO₂",
        log_scale=False,
        show_labels=False  # Cambiar a False para quitar etiquetas numéricas
    )

if "emissions_rate" in df_all.columns:
    # Gráfico de barras original
    plot_grouped_bars(
        data=df_all,
        x_col="config",
        group_col="framework",
        y_col="emissions_rate",
        title="Emissions rate (Server)",
        ylabel="kg CO₂/h",
        y_min_pct=0.85
    )
    
    # Nuevo gráfico con líneas de tendencia
    plot_trends(
        data=df_all,
        y_col="emissions_rate",
        title="Emissions rate (Server)",
        ylabel="kg CO₂/h",
        log_scale=False,
        show_labels=False
    )

if "energy_consumed" in df_all.columns:
    # Gráfico de barras original
    plot_grouped_bars(
        data=df_all,
        x_col="config",
        group_col="framework",
        y_col="energy_consumed",
        title="Energy consumed (Server)",
        ylabel="kWh",
        y_min_pct=0.85
    )
    
    # Nuevo gráfico con líneas de tendencia
    plot_trends(
        data=df_all,
        y_col="energy_consumed",
        title="Energy consumed (Server)",
        ylabel="kWh",
        log_scale=False,
        show_labels=False
    )

if "cpu_energy" in df_all.columns:
    # Gráfico de barras original
    plot_grouped_bars(
        data=df_all,
        x_col="config",
        group_col="framework",
        y_col="cpu_energy",
        title="CPU energy (Server)",
        ylabel="kWh",
        y_min_pct=0.85
    )
    
    # Nuevo gráfico con líneas de tendencia
    plot_trends(
        data=df_all,
        y_col="cpu_energy",
        title="CPU energy (Server)",
        ylabel="kWh",
        log_scale=False,
        show_labels=False
    )

if "ram_energy" in df_all.columns:
    # Gráfico de barras original
    plot_grouped_bars(
        data=df_all,
        x_col="config",
        group_col="framework",
        y_col="ram_energy",
        title="RAM energy (Server)",
        ylabel="kWh",
        y_min_pct=0.85,
        log_scale=True
    )
    
    # Nuevo gráfico con líneas de tendencia
    plot_trends(
        data=df_all,
        y_col="ram_energy",
        title="RAM energy (Server)",
        ylabel="kWh",
        log_scale=True,
        show_labels=False
    )

if "cpu_power" in df_all.columns:
    # Gráfico de barras original
    plot_grouped_bars(
        data=df_all,
        x_col="config",
        group_col="framework",
        y_col="cpu_power",
        title="CPU power (Server)",
        ylabel="W",
        y_min_pct=0.85,
        log_scale=True
    )
    
    # Nuevo gráfico con líneas de tendencia
    plot_trends(
        data=df_all,
        y_col="cpu_power",
        title="CPU power (Server)",
        ylabel="W",
        log_scale=True,
        show_labels=False
    )

if "ram_power" in df_all.columns:
    # Gráfico de barras original
    plot_grouped_bars(
        data=df_all,
        x_col="config",
        group_col="framework",
        y_col="ram_power",
        title="RAM power (Server)",
        ylabel="W",
        y_min_pct=0.85,
        log_scale=True
    )
    
    # Nuevo gráfico con líneas de tendencia
    plot_trends(
        data=df_all,
        y_col="ram_power",
        title="RAM power (Server)",
        ylabel="W",
        log_scale=True,
        show_labels=False
    )

if "duration" in df_all.columns:
    # Gráfico de barras original
    plot_grouped_bars(
        data=df_all,
        x_col="config",
        group_col="framework",
        y_col="duration",
        title="Running time (Server)",
        ylabel="seconds",
        y_min_pct=0.85
    )
    
    # Nuevo gráfico con líneas de tendencia
    plot_trends(
        data=df_all,
        y_col="duration",
        title="Running time (Server)",
        ylabel="seconds",
        log_scale=False,
        show_labels=False
    )

COMPARATIVA ESTADÍSTICAS CODECARBON ORDENADOR VS SERVIDOR

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats

# Configuración de estilo para gráficos más atractivos
plt.style.use('seaborn-v0_8-whitegrid')     # Estilo base de seaborn
plt.rcParams['figure.figsize'] = (14, 7)    # Tamaño más grande
plt.rcParams['font.size'] = 12              # Texto más grande
plt.rcParams['axes.labelsize'] = 14         # Etiquetas más grandes
plt.rcParams['axes.titlesize'] = 16         # Título más grande
plt.rcParams['xtick.labelsize'] = 12        # Etiquetas del eje X más grandes
plt.rcParams['ytick.labelsize'] = 12        # Etiquetas del eje Y más grandes
plt.rcParams['legend.fontsize'] = 12        # Leyenda más grande
plt.rcParams['axes.titlepad'] = 20          # Espacio para el título
plt.rcParams['axes.spines.top'] = False     # Eliminar línea superior
plt.rcParams['axes.spines.right'] = False   # Eliminar línea derecha

# Diccionario con (Framework, Maquina) => fichero
files_all = {
    ("ECJ", "Laptop"): "codecarbon_ECJ_ordenador.xlsx",
    ("ECJ", "Server"): "codecarbon_ECJ_pablo.xlsx",
    ("Inspyred", "Laptop"): "codecarbon_inspyred_ordenador_3.xlsx",
    ("Inspyred", "Server"): "codecarbon_inspyred_pablo_3.xlsx",
    ("DEAP", "Laptop"): "codecarbon_DEAP_ordenador.xlsx",
    ("DEAP", "Server"): "codecarbon_DEAP_pablo.xlsx",
    ("Paradiseo", "Laptop"): "codecarbon_paradiseo_ordenador.xlsx",
    ("Paradiseo", "Server"): "codecarbon_paradiseo_pablo.xlsx"
}

# Columnas de interés para comparar
cols_interes = [
    "duration", 
    "emissions", 
    "emissions_rate",
    "energy_consumed", 
    "cpu_energy", 
    "ram_energy", 
    "cpu_power", 
    "ram_power"
]

# Mapa manual de configuraciones (fila => descripción)
config_map = {
    0: "2^6, 0.01",
    1: "2^6, 0.2",
    2: "2^6, 0.8",
    3: "2^10, 0.01",
    4: "2^10, 0.2",
    5: "2^10, 0.8",
    6: "2^14, 0.01",
    7: "2^14, 0.2",
    8: "2^14, 0.8"
}

# Orden correcto de configuraciones para usar en gráficos
config_order = [config_map[i] for i in range(9)]

# Colores personalizados para cada combinación framework-máquina
# Usamos tonos más oscuros para "Laptop" y más claros para "Server"
color_map = {
    "ECJ (Laptop)": "#1f77b4",          # Azul oscuro
    "ECJ (Server)": "#7ab6e8",          # Azul claro
    "Inspyred (Laptop)": "#ff7f0e",     # Naranja oscuro
    "Inspyred (Server)": "#ffbb78",     # Naranja claro
    "DEAP (Laptop)": "#2ca02c",         # Verde oscuro
    "DEAP (Server)": "#98df8a",         # Verde claro
    "Paradiseo (Laptop)": "#d62728",    # Rojo oscuro
    "Paradiseo (Server)": "#ff9896"     # Rojo claro
}

# Unificamos todos los datos en un solo DataFrame
df_list = []
for (fw_name, machine), file_path in files_all.items():
    temp = pd.read_excel(file_path, sheet_name="MEDIAS")
    
    # Filtramos columnas de interés
    existing_cols = [c for c in cols_interes if c in temp.columns]
    df_temp = temp[existing_cols].copy()
    
    # Añadimos columna con la combinación framework + máquina, p.ej. "ECJ (Laptop)"
    df_temp["fw_machine"] = f"{fw_name} ({machine})"
    df_temp["framework"] = fw_name
    df_temp["machine"] = machine
    
    # Asignamos el índice de fila => config_map
    df_temp.reset_index(drop=True, inplace=True)
    df_temp["config"] = df_temp.index.map(config_map)
    
    df_list.append(df_temp)

# Concatenamos
df_all = pd.concat(df_list, ignore_index=True).fillna(0)

# ──────────────────────────────────────────────────────────────────────
# Función mejorada para  un "grouped bar chart"
# ──────────────────────────────────────────────────────────────────────
def plot_grouped_bars(data, x_col, group_col, y_col, title, ylabel, y_min_pct=0.9, log_scale=False):
    """
    Crea un gráfico de barras agrupadas de la columna 'y_col',
    con 'x_col' en el eje X y cada valor de 'group_col' como barra.
    Destaca la barra con el mayor valor en cada categoría (fila).
    
    Args:
        data: DataFrame con los datos
        x_col: Columna para el eje X (categorías)
        group_col: Columna para agrupar (diferentes barras)
        y_col: Columna con los valores a mostrar
        title: Título del gráfico
        ylabel: Etiqueta del eje Y
        y_min_pct: Porcentaje del valor mínimo para ajustar el eje Y (0-1)
        log_scale: Si es True, usa escala logarítmica en el eje Y
    """
    # 1) Pasamos a formato ancho: filas = config, columnas = fw_machine, valores = y_col
    pivoted = data.pivot(index=x_col, columns=group_col, values=y_col)
    
    # Reordenamos las filas según el orden definido en config_order
    pivoted = pivoted.reindex(config_order)
    
    # Obtenemos los valores mínimos y máximos para ajustar el eje Y
    y_min = pivoted.values.min() * y_min_pct
    y_max = pivoted.values.max() * 1.15  # 15% de margen superior
    
    # Creamos el gráfico con más espacio a la derecha para la leyenda
    fig, ax = plt.subplots(figsize=(14, 7))  # Figura más ancha para acomodar la leyenda
    pivoted.plot(
        kind="bar", 
        ax=ax,
        color=[color_map.get(fw, "#333333") for fw in pivoted.columns],
        width=0.7
    )
    
    # Configuración de escala y límites
    if log_scale:
        plt.yscale('log')  # Escala logarítmica para el eje Y
        # Ajustamos el límite superior para evitar que las etiquetas se salgan
        plt.ylim(top=pivoted.values.max() * 1.3)  # 30% de margen superior
    else:
        plt.ylim(y_min, y_max)
    
    # Título y etiquetas con estilo mejorado
    plt.title(title, fontweight='bold', pad=20)
    plt.xlabel("Configuration", fontweight='bold')
    plt.ylabel(ylabel, fontweight='bold')
    plt.xticks(rotation=30, ha="right")
    
    # Destacamos la barra mayor en cada fila (configuración)
    for row_idx, config_name in enumerate(pivoted.index):
        # Obtenemos la fila con sus 8 fw_machine y valores
        row_values = pivoted.loc[config_name]
        # Nombre de la columna (fw_machine) con el valor máximo en esta fila
        max_col = row_values.idxmax()
        # Valor máximo
        max_value = row_values[max_col]
        # Índice que corresponde a ese fw_machine en pivoted.columns
        col_idx = list(pivoted.columns).index(max_col)
        
        # Localizamos la barra que corresponde al row_idx en ese contenedor
        bar = ax.containers[col_idx][row_idx]
        
        # Ya no resaltamos la barra con borde negro
        # Simplemente dejamos un comentario para recordar que esta es la barra con el valor máximo
        
        # Añadimos etiqueta solo a la barra con el valor máximo
        label_y_pos = max_value * 1.01  # Ligeramente por encima en escala normal
            
        ax.text(
            bar.get_x() + bar.get_width()/2,
            label_y_pos,
            f"{max_value:.4g}",
            ha='center',
            va='bottom',
            fontweight='bold',
            color='black',
            fontsize=11
        )
    
    # Mejoramos la leyenda y la colocamos a la derecha del gráfico
    legend = plt.legend(title="Framework (Computer)", frameon=True, loc='center left', bbox_to_anchor=(1.0, 0.5))
    legend.get_frame().set_facecolor('#F8F8F8')
    legend.get_frame().set_edgecolor('#CCCCCC')
    
    # Añadimos grid para facilitar la lectura
    plt.grid(axis='both', linestyle='--', alpha=0.7)
    
    # Ajustamos el layout teniendo en cuenta el espacio para la leyenda
    plt.tight_layout(rect=[0, 0, 0.85, 1])
    
    plt.show()

# Modificación para plot_trends_by_framework con ajuste polinómico
def plot_trends_by_framework(data, y_col, title, ylabel, log_scale=False, show_labels=True):
    """
    Crea un gráfico de líneas con líneas de tendencia polinómicas para visualizar patrones,
    agrupado por framework y mostrando las dos máquinas para cada uno.
    
    Args:
        data: DataFrame con los datos
        y_col: Columna con los valores a mostrar
        title: Título del gráfico
        ylabel: Etiqueta del eje Y
        log_scale: Si es True, usa escala logarítmica en el eje Y
        show_labels: Si es True, muestra las etiquetas numéricas en los puntos
    """
    # Extraemos los frameworks únicos
    frameworks = data['framework'].unique()
    
    # Creamos una figura con subplots para cada framework
    fig, axes = plt.subplots(2, 2, figsize=(16, 12), sharex=True)
    axes = axes.flatten()  # Convertimos a 1D para fácil indexación
    
    # Extraemos los índices de configuración
    config_indices = list(range(len(config_order)))
    
    # Para cada framework, creamos un subplot
    for i, fw in enumerate(frameworks):
        ax = axes[i]
        fw_data = data[data['framework'] == fw].copy()
        
        # Almacenar los valores R² para incluirlos en la leyenda
        r_squared_values = {}
        
        # Para cada máquina, graficamos una línea
        for machine in ['Laptop', 'Server']:
            machine_data = fw_data[fw_data['machine'] == machine].copy()
            fw_machine = f"{fw} ({machine})"
            
            # Preparamos arrays para valores e índices
            machine_values = []
            machine_indices = []
            
            # Para cada configuración, extraemos el valor si existe
            for j, config in enumerate(config_order):
                config_data = machine_data[machine_data['config'] == config]
                if len(config_data) > 0:
                    val = config_data[y_col].values[0]
                    # Para escala logarítmica, aseguramos que los valores sean positivos
                    if log_scale and (val <= 0 or np.isnan(val)):
                        continue
                    machine_values.append(val)
                    machine_indices.append(j)
            
            # Si no hay suficientes datos, continuamos con la siguiente máquina
            if len(machine_indices) < 3:
                continue
            
            # Dibujamos la línea principal con todos los puntos
            line, = ax.plot(machine_indices, machine_values, 'o-', 
                         label=machine,
                         color=color_map.get(fw_machine),
                         linewidth=2, markersize=8)
            
            # Añadimos etiquetas si está habilitado
            if show_labels:
                for j, val in zip(machine_indices, machine_values):
                    if not np.isnan(val) and (val > 0 or not log_scale):
                        ax.text(j, val * (1.05 if log_scale else 1.01), 
                               f"{val:.4g}", ha='center', va='bottom',
                               color=color_map.get(fw_machine),
                               fontsize=9, fontweight='bold')
            
            # Usamos un ajuste polinómico (grado 2) para capturar cambios no lineales
            if len(machine_indices) >= 3:
                z = np.polyfit(machine_indices, machine_values, 2)
                p = np.poly1d(z)
                
                # Generamos puntos adicionales para suavizar la curva
                x_smooth = np.linspace(min(machine_indices), max(machine_indices), 100)
                y_smooth = p(x_smooth)
                
                # Dibujamos la línea de tendencia suavizada
                ax.plot(x_smooth, y_smooth, '--', 
                       color=color_map.get(fw_machine), 
                       linewidth=1.5, alpha=0.7)
                
                # Calculamos R² para evaluar la calidad del ajuste
                y_pred = p(machine_indices)
                ss_total = np.sum((machine_values - np.mean(machine_values))**2)+0.0000000000001
                ss_residual = np.sum((machine_values - y_pred)**2)
                r_squared = 1 - (ss_residual / ss_total)
                r_squared_values[machine] = r_squared
            else:
                r_squared_values[machine] = float('nan')
        
        # Configuración de escala y límites
        if log_scale:
            ax.set_yscale('log')
        
        # Título y etiquetas
        ax.set_title(f"{fw}", fontweight='bold')
        ax.set_xticks(config_indices)
        ax.set_xticklabels(config_order, rotation=30, ha='right')
        if i % 2 == 0:  # Solo para los subplots de la izquierda
            ax.set_ylabel(ylabel, fontweight='bold')
        if i >= 2:  # Solo para los subplots de abajo
            ax.set_xlabel("Configuration", fontweight='bold')
        
        # Añadir áreas sombreadas para separar tamaños de población
        pop_sizes = ["2^6", "2^10", "2^14"]
        colors = ["#f0f0f0", "#e8e8e8", "#f0f0f0"]
        
        for p_idx, pop in enumerate(pop_sizes):
            pop_configs = [j for j, c in enumerate(config_order) if c.startswith(pop)]
            if pop_configs:
                start, end = min(pop_configs), max(pop_configs)
                ax.axvspan(start - 0.5, end + 0.5, alpha=0.2, color=colors[p_idx])
        
        # Crear etiquetas personalizadas para la leyenda que incluyan R²
        custom_labels = []
        custom_handles = []
        
        # Obtenemos los handles actuales y sus etiquetas
        handles, labels = ax.get_legend_handles_labels()
        
        for handle, label in zip(handles, labels):
            if label in r_squared_values:
                r2 = r_squared_values[label]
                if not np.isnan(r2):
                    # Incluimos el valor R² en la etiqueta
                    custom_labels.append(f"{label} (R²={r2:.2f})")
                else:
                    custom_labels.append(label)
                custom_handles.append(handle)
        
        # Crear la leyenda con las etiquetas personalizadas
        ax.legend(custom_handles, custom_labels, title="Computer", frameon=True)
        
        # Grid
        ax.grid(axis='both', linestyle='--', alpha=0.7)
    
    # Título general
    fig.suptitle(f"{title} with Trend Lines", fontsize=18, fontweight='bold', y=0.98)
    
    plt.tight_layout()
    plt.subplots_adjust(top=0.92)
    plt.savefig(f"GRAFICAS/{title} with Trend Lines.png", dpi=300, bbox_inches='tight')
    plt.show()

# Nueva función para mostrar líneas con tendencias por máquina con ajuste polinómico
def plot_trends_by_machine(data, y_col, title, ylabel, log_scale=False, show_labels=True):
    """
    Crea gráficos de líneas con líneas de tendencia polinómicas para visualizar patrones,
    separando los datos por máquina.
    
    Args:
        data: DataFrame con los datos
        y_col: Columna con los valores a mostrar
        title: Título del gráfico
        ylabel: Etiqueta del eje Y
        log_scale: Si es True, usa escala logarítmica en el eje Y
        show_labels: Si es True, muestra las etiquetas numéricas en los puntos
    """
    # Para cada máquina, creamos un gráfico separado
    for machine in ['Laptop', 'Server']:
        machine_data = data[data['machine'] == machine].copy()
        
        # Creamos una figura
        fig, ax = plt.subplots(figsize=(14, 7))
        
        # Extraemos los frameworks únicos
        frameworks = machine_data['framework'].unique()
        
        # Extraemos los índices de configuración
        config_indices = list(range(len(config_order)))
        
        # Almacenar los valores R² para incluirlos en la leyenda
        r_squared_values = {}
        
        # Para cada framework, graficamos una línea
        for fw in frameworks:
            fw_data = machine_data[machine_data['framework'] == fw].copy()
            fw_machine = f"{fw} ({machine})"
            
            # Preparamos arrays para valores e índices
            fw_values = []
            fw_indices = []
            
            # Para cada configuración, extraemos el valor si existe
            for j, config in enumerate(config_order):
                config_data = fw_data[fw_data['config'] == config]
                if len(config_data) > 0:
                    val = config_data[y_col].values[0]
                    # Para escala logarítmica, aseguramos que los valores sean positivos
                    if log_scale and (val <= 0 or np.isnan(val)):
                        continue
                    fw_values.append(val)
                    fw_indices.append(j)
            
            # Si no hay suficientes datos, continuamos con el siguiente framework
            if len(fw_indices) < 3:
                continue
            
            # Dibujamos la línea principal con todos los puntos
            line, = ax.plot(fw_indices, fw_values, 'o-', 
                         label=fw,
                         color=color_map.get(fw_machine),
                         linewidth=2, markersize=8)
            
            # Añadimos etiquetas si está habilitado
            if show_labels:
                for j, val in zip(fw_indices, fw_values):
                    if not np.isnan(val) and (val > 0 or not log_scale):
                        ax.text(j, val * (1.05 if log_scale else 1.01), 
                               f"{val:.4g}", ha='center', va='bottom',
                               color=color_map.get(fw_machine),
                               fontsize=9, fontweight='bold')
            
            # Usamos un ajuste polinómico (grado 2) para capturar cambios no lineales
            if len(fw_indices) >= 3:
                z = np.polyfit(fw_indices, fw_values, 2)
                p = np.poly1d(z)
                
                # Generamos puntos adicionales para suavizar la curva
                x_smooth = np.linspace(min(fw_indices), max(fw_indices), 100)
                y_smooth = p(x_smooth)
                
                # Dibujamos la línea de tendencia suavizada
                ax.plot(x_smooth, y_smooth, '--', 
                       color=color_map.get(fw_machine), 
                       linewidth=1.5, alpha=0.7)
                
                # Calculamos R² para evaluar la calidad del ajuste
                y_pred = p(fw_indices)
                ss_total = np.sum((fw_values - np.mean(fw_values))**2)+0.0000000000001
                ss_residual = np.sum((fw_values - y_pred)**2)
                r_squared = 1 - (ss_residual / ss_total)
                r_squared_values[fw] = r_squared
            else:
                r_squared_values[fw] = float('nan')
        
        # Configuración de escala y límites
        if log_scale:
            ax.set_yscale('log')
        
        # Título y etiquetas
        ax.set_title(f"{title} - {machine}", fontweight='bold', pad=20)
        ax.set_xticks(config_indices)
        ax.set_xticklabels(config_order, rotation=30, ha='right')
        ax.set_xlabel("Configuration", fontweight='bold')
        ax.set_ylabel(ylabel, fontweight='bold')
        
        # Añadir áreas sombreadas para separar tamaños de población
        pop_sizes = ["2^6", "2^10", "2^14"]
        colors = ["#f0f0f0", "#e8e8e8", "#f0f0f0"]
        
        for p_idx, pop in enumerate(pop_sizes):
            pop_configs = [j for j, c in enumerate(config_order) if c.startswith(pop)]
            if pop_configs:
                start, end = min(pop_configs), max(pop_configs)
                ax.axvspan(start - 0.5, end + 0.5, alpha=0.2, color=colors[p_idx])
                ax.text((start + end) / 2, 0.98, 
                       f"Population {pop}", ha='center', fontsize=10, alpha=0.7,
                       transform=ax.get_xaxis_transform())
        
        # Crear etiquetas personalizadas para la leyenda que incluyan R²
        custom_labels = []
        custom_handles = []
        
        # Obtenemos los handles actuales y sus etiquetas
        handles, labels = ax.get_legend_handles_labels()
        
        for handle, label in zip(handles, labels):
            if label in r_squared_values:
                r2 = r_squared_values[label]
                if not np.isnan(r2):
                    # Incluimos el valor R² en la etiqueta
                    custom_labels.append(f"{label} (R²={r2:.2f})")
                else:
                    custom_labels.append(label)
                custom_handles.append(handle)
        
        # Crear la leyenda con las etiquetas personalizadas
        legend = ax.legend(custom_handles, custom_labels, title="Framework", frameon=True, 
                          loc='center left', bbox_to_anchor=(1.0, 0.5))
        legend.get_frame().set_facecolor('#F8F8F8')
        legend.get_frame().set_edgecolor('#CCCCCC')
        
        # Grid y layout
        ax.grid(axis='both', linestyle='--', alpha=0.7)
        plt.tight_layout(rect=[0, 0, 0.85, 1])
        
        plt.show()
# ─────────────────────────────────────────────────────────────────────────
# Generamos gráficos de barras y de tendencias para cada columna
# ─────────────────────────────────────────────────────────────────────────
if "emissions" in df_all.columns:
    # Gráfico de barras original
    plot_grouped_bars(
        data=df_all,
        x_col="config",
        group_col="fw_machine",
        y_col="emissions",
        title="CO₂ Emissions: Laptop vs Server",
        ylabel="kg CO₂",
        y_min_pct=0.85,
        log_scale=True
    )
    
    # Nuevos gráficos con líneas de tendencia
    plot_trends_by_framework(
        data=df_all,
        y_col="emissions",
        title="CO₂ Emissions",
        ylabel="kg CO₂",
        log_scale=True,
        show_labels=False  # Cambiar a False para quitar etiquetas numéricas
    )
    
    plot_trends_by_machine(
        data=df_all,
        y_col="emissions",
        title="CO₂ Emissions",
        ylabel="kg CO₂",
        log_scale=True,
        show_labels=False  # Cambiar a False para quitar etiquetas numéricas
    )

if "emissions_rate" in df_all.columns:
    # Gráfico de barras original
    plot_grouped_bars(
        data=df_all,
        x_col="config",
        group_col="fw_machine",
        y_col="emissions_rate",
        title="Emissions rate: Laptop vs Server",
        ylabel="kg CO₂/h",
        y_min_pct=0.85,
        log_scale=True
    )
    
    # Nuevos gráficos con líneas de tendencia
    plot_trends_by_framework(
        data=df_all,
        y_col="emissions_rate",
        title="Emissions Rate",
        ylabel="kg CO₂/h",
        log_scale=True,
        show_labels=False
    )
    
    plot_trends_by_machine(
        data=df_all,
        y_col="emissions_rate",
        title="Emissions Rate",
        ylabel="kg CO₂/h",
        log_scale=True,
        show_labels=False
    )

if "energy_consumed" in df_all.columns:
    # Gráfico de barras original
    plot_grouped_bars(
        data=df_all,
        x_col="config",
        group_col="fw_machine",
        y_col="energy_consumed",
        title="Consumed energy: Laptop vs Server",
        ylabel="kWh",
        y_min_pct=0.85,
        log_scale=True
    )
    
    # Nuevos gráficos con líneas de tendencia
    plot_trends_by_framework(
        data=df_all,
        y_col="energy_consumed",
        title="Energy Consumed",
        ylabel="kWh",
        log_scale=True,
        show_labels=False
    )
    
    plot_trends_by_machine(
        data=df_all,
        y_col="energy_consumed",
        title="Energy Consumed",
        ylabel="kWh",
        log_scale=True,
        show_labels=False
    )

if "cpu_energy" in df_all.columns:
    # Gráfico de barras original
    plot_grouped_bars(
        data=df_all,
        x_col="config",
        group_col="fw_machine",
        y_col="cpu_energy",
        title="CPU energy: Laptop vs Server",
        ylabel="kWh",
        y_min_pct=0.85,
        log_scale=True
    )
    
    # Nuevos gráficos con líneas de tendencia
    plot_trends_by_framework(
        data=df_all,
        y_col="cpu_energy",
        title="CPU Energy",
        ylabel="kWh",
        log_scale=True,
        show_labels=False
    )
    
    plot_trends_by_machine(
        data=df_all,
        y_col="cpu_energy",
        title="CPU Energy",
        ylabel="kWh",
        log_scale=True,
        show_labels=False
    )

if "ram_energy" in df_all.columns:
    # Gráfico de barras original
    plot_grouped_bars(
        data=df_all,
        x_col="config",
        group_col="fw_machine",
        y_col="ram_energy",
        title="RAM energy: Laptop vs Server",
        ylabel="kWh",
        y_min_pct=0.85,
        log_scale=True
    )
    
    # Nuevos gráficos con líneas de tendencia
    plot_trends_by_framework(
        data=df_all,
        y_col="ram_energy",
        title="RAM Energy",
        ylabel="kWh",
        log_scale=True,
        show_labels=False
    )
    
    plot_trends_by_machine(
        data=df_all,
        y_col="ram_energy",
        title="RAM Energy",
        ylabel="kWh",
        log_scale=True,
        show_labels=False
    )

if "cpu_power" in df_all.columns:
    # Gráfico de barras original
    plot_grouped_bars(
        data=df_all,
        x_col="config",
        group_col="fw_machine",
        y_col="cpu_power",
        title="CPU power: Laptop vs Server",
        ylabel="W",
        y_min_pct=0.85,
        log_scale=True
    )
    
    # Nuevos gráficos con líneas de tendencia
    plot_trends_by_framework(
        data=df_all,
        y_col="cpu_power",
        title="CPU Power",
        ylabel="W",
        log_scale=True,
        show_labels=False
    )
    
    plot_trends_by_machine(
        data=df_all,
        y_col="cpu_power",
        title="CPU Power",
        ylabel="W",
        log_scale=True,
        show_labels=False
    )

if "ram_power" in df_all.columns:
    # Gráfico de barras original
    plot_grouped_bars(
        data=df_all,
        x_col="config",
        group_col="fw_machine",
        y_col="ram_power",
        title="RAM power: Laptop vs Server",
        ylabel="W",
        y_min_pct=0.85,
        log_scale=True
    )
    
    # Nuevos gráficos con líneas de tendencia
    plot_trends_by_framework(
        data=df_all,
        y_col="ram_power",
        title="RAM Power",
        ylabel="W",
        log_scale=True,
        show_labels=False
    )
    
    plot_trends_by_machine(
        data=df_all,
        y_col="ram_power",
        title="RAM Power",
        ylabel="W",
        log_scale=True,
        show_labels=False
    )
    
if "duration" in df_all.columns:
    # Gráfico de barras original
    plot_grouped_bars(
        data=df_all,
        x_col="config",
        group_col="fw_machine",
        y_col="duration",
        title="Running time: Laptop vs Server",
        ylabel="seconds",
        y_min_pct=0.85
    )
    
    # Nuevos gráficos con líneas de tendencia
    plot_trends_by_framework(
        data=df_all,
        y_col="duration",
        title="Running Time",
        ylabel="seconds",
        log_scale=False,
        show_labels=False
    )
    
    plot_trends_by_machine(
        data=df_all,
        y_col="duration",
        title="Running Time",
        ylabel="seconds",
        log_scale=False,
        show_labels=False
    )    

ESTADÍSTICAS RESULTADOS EN MI ORDENADOR

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats

# Configuración de estilo para gráficos más atractivos
plt.style.use('seaborn-v0_8-whitegrid')     # Estilo base de seaborn
plt.rcParams['figure.figsize'] = (14, 7)    # Tamaño más grande
plt.rcParams['font.size'] = 12              # Texto más grande
plt.rcParams['axes.labelsize'] = 14         # Etiquetas más grandes
plt.rcParams['axes.titlesize'] = 16         # Título más grande
plt.rcParams['xtick.labelsize'] = 12        # Etiquetas del eje X más grandes
plt.rcParams['ytick.labelsize'] = 12        # Etiquetas del eje Y más grandes
plt.rcParams['legend.fontsize'] = 12        # Leyenda más grande
plt.rcParams['axes.titlepad'] = 20          # Espacio para el título
plt.rcParams['axes.spines.top'] = False     # Eliminar línea superior
plt.rcParams['axes.spines.right'] = False   # Eliminar línea derecha

# Archivos de resultados del Laptop
files_resultados_ordenador = {
    "ECJ": "resultados_ECJ_ordenador.xlsx",
    "Inspyred": "resultados_inspyred_ordenador_3.xlsx",
    "DEAP": "resultados_DEAP_ordenador.xlsx",
    "Paradiseo": "resultados_paradiseo_ordenador.xlsx"
}

# Columnas de interés para comparar resultados
cols_resultados = [
    "generacion", 
    "fitness_inicial",
    "variacion_fitness",
    "fitness_maximo",
    "tiempo_transcurrido"
]

# Mapa manual de configuraciones
config_map = {
    0: "2^6, 0.01",
    1: "2^6, 0.2",
    2: "2^6, 0.8",
    3: "2^10, 0.01",
    4: "2^10, 0.2",
    5: "2^10, 0.8",
    6: "2^14, 0.01",
    7: "2^14, 0.2",
    8: "2^14, 0.8",
}

# Orden correcto de configuraciones para usar en gráficos
config_order = [config_map[i] for i in range(9)]

# Colores personalizados para cada framework
framework_colors = {
    "ECJ": "#1f77b4",       # Azul
    "Inspyred": "#ff7f0e",  # Naranja
    "DEAP": "#2ca02c",      # Verde
    "Paradiseo": "#d62728"  # Rojo
}

# Leemos y unificamos todos los datos
df_list = []
for fw_name, file_path in files_resultados_ordenador.items():
    temp = pd.read_excel(file_path, sheet_name="MEDIAS")
    
    # Filtramos solo columnas relevantes si existen
    existing_cols = [c for c in cols_resultados if c in temp.columns]
    df_temp = temp[existing_cols].copy()
    
    # Añadimos el nombre del framework
    df_temp["framework"] = fw_name
    
    # Asignamos configuración usando el índice
    df_temp.reset_index(drop=True, inplace=True)
    df_temp["config"] = df_temp.index.map(config_map)
    
    df_list.append(df_temp)

# Concatenamos en un solo DataFrame
df_resultados = pd.concat(df_list, ignore_index=True).fillna(0)

# ───────────────────────────────────────────────────────────
# Función mejorada para hacer gráficas agrupadas
# ───────────────────────────────────────────────────────────
def plot_grouped_bars(data, x_col, group_col, y_col, title, ylabel, y_min_pct=0.9, log_scale=False):
    """
    Crea un gráfico de barras agrupadas de la columna 'y_col',
    con 'x_col' en el eje X y cada valor de 'group_col' como una barra.
    
    Args:
        data: DataFrame con los datos
        x_col: Columna para el eje X (categorías)
        group_col: Columna para agrupar (diferentes barras)
        y_col: Columna con los valores a mostrar
        title: Título del gráfico
        ylabel: Etiqueta del eje Y
        y_min_pct: Porcentaje del valor mínimo para ajustar el eje Y (0-1)
        log_scale: Si es True, usa escala logarítmica en el eje Y
    """
    # 1) Pasamos a formato ancho: filas = config, columnas = framework, valores = y_col
    pivoted = data.pivot(index=x_col, columns=group_col, values=y_col)
    
    # Reordenamos las filas según el orden definido en config_order
    pivoted = pivoted.reindex(config_order)
    
    # Obtenemos los valores mínimos y máximos para ajustar el eje Y
    y_min = pivoted.values.min() * y_min_pct
    y_max = pivoted.values.max() * 1.15  # 15% de margen superior
    
    # Creamos el gráfico con más espacio a la derecha para la leyenda
    fig, ax = plt.subplots(figsize=(14, 7))  # Figura más ancha para acomodar la leyenda
    pivoted.plot(
        kind="bar", 
        ax=ax,
        color=[framework_colors.get(fw, "#333333") for fw in pivoted.columns],
        width=0.7
    )
    
    # Configuración de escala y límites
    if log_scale:
        plt.yscale('log')  # Escala logarítmica para el eje Y
        # Ajustamos el límite superior para evitar que las etiquetas se salgan
        plt.ylim(top=pivoted.values.max() * 1.3)  # 30% de margen superior
    else:
        plt.ylim(y_min, y_max)
    
    # Título y etiquetas con estilo mejorado
    plt.title(title, fontweight='bold', pad=20)
    plt.xlabel("Configuration", fontweight='bold')
    plt.ylabel(ylabel, fontweight='bold')
    plt.xticks(rotation=30, ha="right")
    
    # Destacamos la barra mayor en cada fila (configuración)
    for row_idx, config_name in enumerate(pivoted.index):
        # Obtenemos la fila con sus frameworks y valores
        row_values = pivoted.loc[config_name]
        # Nombre de la columna (framework) con el valor máximo en esta fila
        max_col = row_values.idxmax()
        # Valor máximo
        max_value = row_values[max_col]
        # Índice que corresponde a ese framework en pivoted.columns
        col_idx = list(pivoted.columns).index(max_col)
        
        # Localizamos la barra que corresponde al row_idx en ese contenedor
        bar = ax.containers[col_idx][row_idx]
        
        # Ya no resaltamos la barra con borde negro
        # Simplemente dejamos un comentario para recordar que esta es la barra con el valor máximo
        
        # Añadimos etiqueta solo a la barra con el valor máximo
        label_y_pos = max_value * 1.01  # Ligeramente por encima en escala normal
            
        ax.text(
            bar.get_x() + bar.get_width()/2,
            label_y_pos,
            f"{max_value:.4g}",
            ha='center',
            va='bottom',
            fontweight='bold',
            color='black',
            fontsize=11
        )
    
    # Mejoramos la leyenda y la colocamos a la derecha del gráfico
    legend = plt.legend(title="Framework", frameon=True, loc='center left', bbox_to_anchor=(1.0, 0.5))
    legend.get_frame().set_facecolor('#F8F8F8')
    legend.get_frame().set_edgecolor('#CCCCCC')
    
    # Añadimos grid para facilitar la lectura
    plt.grid(axis='both', linestyle='--', alpha=0.7)
    
    # Ajustamos el layout teniendo en cuenta el espacio para la leyenda
    plt.tight_layout(rect=[0, 0, 0.85, 1])
    
    plt.show()

# Función mejorada para mostrar líneas con tendencias adaptativas
def plot_trends(data, y_col, title, ylabel, log_scale=False, show_labels=True):
    """
    Crea un gráfico de líneas con líneas de tendencia adaptativas que se ajustan mejor
    a datos con cambios bruscos.
    
    Args:
        data: DataFrame con los datos
        y_col: Columna con los valores a mostrar
        title: Título del gráfico
        ylabel: Etiqueta del eje Y
        log_scale: Si es True, usa escala logarítmica en el eje Y
        show_labels: Si es True, muestra las etiquetas numéricas en los puntos
    """
    # Creamos una figura
    fig, ax = plt.subplots(figsize=(14, 7))
    
    # Extraemos los frameworks únicos
    frameworks = data['framework'].unique()
    
    # Extraemos los índices de configuración para representar mejor la tendencia
    config_indices = list(range(len(config_order)))
    
    # Almacenar los valores R² para incluirlos en la leyenda
    r_squared_values = {}
    
    # Para cada framework, graficamos una línea
    for fw in frameworks:
        fw_data = data[data['framework'] == fw].copy()
        
        # Preparamos arrays para valores e índices
        fw_values = []
        fw_indices = []
        
        # Para cada configuración, extraemos el valor si existe
        for i, config in enumerate(config_order):
            config_data = fw_data[fw_data['config'] == config]
            if len(config_data) > 0:
                val = config_data[y_col].values[0]
                # Para escala logarítmica, aseguramos que los valores sean positivos
                if log_scale and (val <= 0 or np.isnan(val)):
                    continue
                fw_values.append(val)
                fw_indices.append(i)
                
        # Si no hay suficientes datos, continuamos con el siguiente framework
        if len(fw_indices) < 3:
            continue
        
        # Dibujamos la línea principal con todos los puntos
        line, = ax.plot(fw_indices, fw_values, 'o-', 
                      label=fw,
                      color=framework_colors.get(fw),
                      linewidth=2, markersize=8)
        
        # Añadimos etiquetas si está habilitado
        if show_labels:
            for i, val in zip(fw_indices, fw_values):
                if not np.isnan(val) and (val > 0 or not log_scale):
                    ax.text(i, val * (1.05 if log_scale else 1.01), 
                           f"{val:.4g}", ha='center', va='bottom',
                           color=framework_colors.get(fw),
                           fontsize=9, fontweight='bold')
        
        # Generamos más puntos para suavizar la línea de tendencia
        if len(fw_indices) >= 3:
            # Usamos un ajuste polinómico (grado 2) para capturar cambios no lineales
            # El grado 2 permite capturar una curvatura básica, suficiente para estos datos
            z = np.polyfit(fw_indices, fw_values, 2)
            p = np.poly1d(z)
            
            # Generamos puntos adicionales para suavizar la curva
            x_smooth = np.linspace(min(fw_indices), max(fw_indices), 100)
            y_smooth = p(x_smooth)
            
            # Dibujamos la línea de tendencia suavizada
            ax.plot(x_smooth, y_smooth, '--', 
                   color=framework_colors.get(fw), 
                   linewidth=1.5, alpha=0.7)
            
            # Calculamos R² para evaluar la calidad del ajuste
            y_pred = p(fw_indices)
            ss_total = np.sum((fw_values - np.mean(fw_values))**2)+0.0000000000001
            ss_residual = np.sum((fw_values - y_pred)**2)
            r_squared = 1 - (ss_residual / ss_total)
            r_squared_values[fw] = r_squared
        else:
            r_squared_values[fw] = float('nan')
    
    # Configuración de escala y límites
    if log_scale:
        ax.set_yscale('log')
    
    # Título y etiquetas
    ax.set_title(f"{title} with Trend Lines", fontweight='bold', pad=20)
    ax.set_xticks(config_indices)
    ax.set_xticklabels(config_order, rotation=30, ha='right')
    ax.set_xlabel("Configuration", fontweight='bold')
    ax.set_ylabel(ylabel, fontweight='bold')
    
    # Añadir áreas sombreadas para separar tamaños de población
    pop_sizes = ["2^6", "2^10", "2^14"]
    colors = ["#f0f0f0", "#e8e8e8", "#f0f0f0"]
    
    for p_idx, pop in enumerate(pop_sizes):
        pop_configs = [i for i, c in enumerate(config_order) if c.startswith(pop)]
        if pop_configs:
            start, end = min(pop_configs), max(pop_configs)
            ax.axvspan(start - 0.5, end + 0.5, alpha=0.2, color=colors[p_idx])
            ax.text((start + end) / 2, 0.98, 
                   f"Population {pop}", ha='center', fontsize=10, alpha=0.7,
                   transform=ax.get_xaxis_transform())
    
    # Crear etiquetas personalizadas para la leyenda que incluyan R²
    custom_labels = []
    custom_handles = []
    
    # Obtenemos los handles actuales y sus etiquetas
    handles, labels = ax.get_legend_handles_labels()
    
    for handle, label in zip(handles, labels):
        if label in r_squared_values:
            r2 = r_squared_values[label]
            if not np.isnan(r2):
                # Incluimos el valor R² en la etiqueta
                custom_labels.append(f"{label} (R²={r2:.2f})")
            else:
                custom_labels.append(label)
            custom_handles.append(handle)
    
    # Crear la leyenda con las etiquetas personalizadas
    legend = ax.legend(custom_handles, custom_labels, title="Framework", 
                      frameon=True, loc='center left', bbox_to_anchor=(1.0, 0.5))
    legend.get_frame().set_facecolor('#F8F8F8')
    legend.get_frame().set_edgecolor('#CCCCCC')
    
    # Grid y layout
    ax.grid(axis='both', linestyle='--', alpha=0.7)
    plt.tight_layout(rect=[0, 0, 0.85, 1])
    
    plt.show()
# ───────────────────────────────────────────────────────────
# Hacemos gráficas de barras y de tendencias para cada métrica
# ───────────────────────────────────────────────────────────
if "generacion" in df_resultados.columns:
    # Gráfico de barras original
    plot_grouped_bars(
        data=df_resultados,
        x_col="config",
        group_col="framework",
        y_col="generacion",
        title="Generations reached",
        ylabel="Generations",
        y_min_pct=0.85,
        log_scale=True
    )
    
    # Gráfico con líneas de tendencia
    plot_trends(
        data=df_resultados,
        y_col="generacion",
        title="Generations reached",
        ylabel="Generations",
        log_scale=True,
        show_labels=False
    )

if "fitness_inicial" in df_resultados.columns:
    # Gráfico de barras original
    plot_grouped_bars(
        data=df_resultados,
        x_col="config",
        group_col="framework",
        y_col="fitness_inicial",
        title="Initial Fitness",
        ylabel="Initial Fitness",
        y_min_pct=0.85
    )
    
    # Gráfico con líneas de tendencia
    plot_trends(
        data=df_resultados,
        y_col="fitness_inicial",
        title="Initial Fitness",
        ylabel="Initial Fitness",
        log_scale=False,
        show_labels=False
    )

if "variacion_fitness" in df_resultados.columns:
    # Gráfico de barras original
    plot_grouped_bars(
        data=df_resultados,
        x_col="config",
        group_col="framework",
        y_col="variacion_fitness",
        title="Fitness Variation",
        ylabel="Fitness Variation",
        y_min_pct=0.85
    )
    
    # Gráfico con líneas de tendencia
    plot_trends(
        data=df_resultados,
        y_col="variacion_fitness",
        title="Fitness Variation",
        ylabel="Fitness Variation",
        log_scale=False,
        show_labels=False
    )

if "fitness_maximo" in df_resultados.columns:
    # Gráfico de barras original
    plot_grouped_bars(
        data=df_resultados,
        x_col="config",
        group_col="framework",
        y_col="fitness_maximo",
        title="Maximum fitness reached",
        ylabel="Maximum fitness",
        y_min_pct=0.85
    )
    
    # Gráfico con líneas de tendencia
    plot_trends(
        data=df_resultados,
        y_col="fitness_maximo",
        title="Maximum fitness reached",
        ylabel="Maximum fitness",
        log_scale=False,
        show_labels=False
    )

if "tiempo_transcurrido" in df_resultados.columns:
    # Gráfico de barras original
    plot_grouped_bars(
        data=df_resultados,
        x_col="config",
        group_col="framework",
        y_col="tiempo_transcurrido",
        title="Elapsed time",
        ylabel="Time (ms)",
        y_min_pct=0.85,
        log_scale=True
    )
    
    # Gráfico con líneas de tendencia
    plot_trends(
        data=df_resultados,
        y_col="tiempo_transcurrido",
        title="Elapsed time",
        ylabel="Time (ms)",
        log_scale=True,
        show_labels=False
    )

diferentes comparativas de los mismos datos RESULTADOS EN MI ORDENADOR

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats

# Add a global control variable to show or hide value labels
SHOW_VALUE_LABELS = False  # Set to False to hide all value labels on charts

# Configuración de estilo para gráficos más atractivos
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = [14, 8]
plt.rcParams['font.size'] = 12
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['legend.fontsize'] = 12
plt.rcParams['axes.titlepad'] = 20
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False

# English translations for metric names
metric_translations = {
    "fitness_inicial": "initial_fitness",
    "variacion_fitness": "fitness_variation",
    "fitness_maximo": "maximum_fitness",
    "generacion": "generation",
    "tiempo_transcurrido": "elapsed_time"
}

# Archivos de resultados
files_resultados_ordenador = {
    "ECJ": "resultados_ECJ_ordenador.xlsx",
    "Inspyred": "resultados_inspyred_ordenador_3.xlsx",
    "DEAP": "resultados_DEAP_ordenador.xlsx",
    "Paradiseo": "resultados_paradiseo_ordenador.xlsx"
}
# Columnas de interés para comparar resultados
cols_resultados = [
    "fitness_inicial",
    "variacion_fitness",
    "fitness_maximo",  
    "generacion",
    "tiempo_transcurrido"
]

# Mapa manual de configuraciones
config_map = {
    0: "2^6, 0.01",
    1: "2^6, 0.2",
    2: "2^6, 0.8",
    3: "2^10, 0.01",
    4: "2^10, 0.2",
    5: "2^10, 0.8",
    6: "2^14, 0.01",
    7: "2^14, 0.2",
    8: "2^14, 0.8",
}

# Orden correcto de configuraciones
config_order = [config_map[i] for i in range(9)]

# Colores personalizados para cada framework
framework_colors = {
    "ECJ": "#1f77b4",       # Azul
    "Inspyred": "#ff7f0e",  # Naranja
    "DEAP": "#2ca02c",      # Verde
    "Paradiseo": "#d62728"  # Rojo
}

# Colores para las diferentes métricas
metric_colors = {
    "fitness_inicial": "#3498db",    # Azul claro
    "variacion_fitness": "#2ecc71",  # Verde claro
    "fitness_maximo": "#e74c3c"      # Rojo claro
}

# Leemos y unificamos todos los datos
df_list = []
for fw_name, file_path in files_resultados_ordenador.items():
    temp = pd.read_excel(file_path, sheet_name="MEDIAS")
    
    # Filtramos solo columnas relevantes si existen
    existing_cols = [c for c in cols_resultados if c in temp.columns]
    df_temp = temp[existing_cols].copy()
    
    # Añadimos el nombre del framework
    df_temp["framework"] = fw_name
    
    # Asignamos configuración usando el índice
    df_temp.reset_index(drop=True, inplace=True)
    df_temp["config"] = df_temp.index.map(config_map)
    
    df_list.append(df_temp)

# Concatenamos en un solo DataFrame
df_resultados = pd.concat(df_list, ignore_index=True).fillna(0)

# Función para crear gráficas de barras agrupadas con múltiples métricas
def plot_metrics_comparison(data, config_values, frameworks, metrics, title):
    """
    Creates a chart showing multiple metrics for each framework in a specific configuration.
    
    Args:
        data: DataFrame with the data
        config_values: List of configuration values to include
        frameworks: List of frameworks to compare
        metrics: List of metrics to show for each framework
        title: Chart title
    """
    # Filtrar datos para las configuraciones específicas
    filtered_data = data[data['config'].isin(config_values)].copy()
    
    # Creamos una figura con subplots para cada configuración
    n_configs = len(config_values)
    fig, axes = plt.subplots(1, n_configs, figsize=(18, 6), sharey=True)
    
    # Si solo hay una configuración, axes no será un array, lo convertimos
    if n_configs == 1:
        axes = [axes]
    
    # Para cada configuración
    for i, config in enumerate(config_values):
        ax = axes[i]
        config_data = filtered_data[filtered_data['config'] == config]
        
        # Para cada framework, creamos un grupo de barras (una por métrica)
        bar_width = 0.8 / len(frameworks)
        
        for j, framework in enumerate(frameworks):
            fw_data = config_data[config_data['framework'] == framework]
            if len(fw_data) == 0:
                continue
                
            x_positions = np.arange(len(metrics))
            values = [fw_data[metric].values[0] if not fw_data[metric].empty else 0 for metric in metrics]
            
            # Posición de las barras de este framework
            x_offset = j * bar_width - (len(frameworks) - 1) * bar_width / 2
            
            # Dibujamos las barras para este framework
            bars = ax.bar(x_positions + x_offset, values, width=bar_width, 
                         label=framework if i == 0 else "", 
                         color=framework_colors.get(framework))
            
            # Añadimos etiquetas de valores solo si SHOW_VALUE_LABELS es True
            if SHOW_VALUE_LABELS:
                for k, bar in enumerate(bars):
                    if values[k] > 0.01:  # Solo mostramos etiquetas para valores significativos
                        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                               f"{values[k]:.3f}", ha='center', va='bottom', 
                               fontsize=8, fontweight='bold', rotation=45)
        
        # Configuramos cada subplot
        ax.set_title(f"Configuration: {config}", fontsize=12, fontweight='bold')
        ax.set_xticks(range(len(metrics)))
        
        # Use English names for metrics
        english_metrics = [metric_translations.get(m, m).replace('_', ' ').title() for m in metrics]
        ax.set_xticklabels(english_metrics, rotation=45, ha='right')
        ax.grid(axis='both', linestyle='--', alpha=0.7)
        
        # Para el primer subplot, añadimos etiqueta del eje Y
        if i == 0:
            ax.set_ylabel("Value", fontweight='bold')
    
    # Título general
    fig.suptitle(title, fontsize=16, fontweight='bold', y=1.05)
    
    # Leyenda única para toda la figura
    handles, labels = axes[0].get_legend_handles_labels()
    fig.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, 0), 
              ncol=len(frameworks), frameon=True, fancybox=True, shadow=True)
    
    plt.tight_layout()
    plt.subplots_adjust(bottom=0.2)  # Espacio para la leyenda
    
    plt.show()

# Función para crear gráficas por tamaño de población
def plot_by_population_size(data, pop_size, metrics, title):
    """
    Creates charts for configurations with a specific population size.
    
    Args:
        data: DataFrame with the data
        pop_size: Population size ("2^6", "2^10", "2^14")
        metrics: List of metrics to show
        title: Base title for the chart
    """
    # Filtramos configuraciones para este tamaño de población
    configs = [c for c in config_order if c.startswith(pop_size)]
    
    # Frameworks disponibles
    frameworks = data['framework'].unique()
    
    plot_metrics_comparison(
        data=data,
        config_values=configs,
        frameworks=frameworks,
        metrics=metrics,
        title=f"{title} (Population {pop_size})"
    )

# Definición de la función plot_framework_metrics antes de usarla
def plot_framework_metrics(data, framework_name, metrics, title):
    """
    Creates a chart showing the metrics of a specific framework across all configurations.
    
    Args:
        data: DataFrame with the data
        framework_name: Name of the framework to analyze
        metrics: List of metrics to show
        title: Chart title
    """
    # Filtrar datos solo para el framework especificado
    fw_data = data[data['framework'] == framework_name].copy()
    
    # Preparar figura
    fig, ax = plt.subplots(figsize=(14, 7))
    
    # Número de grupos (configuraciones) y de barras por grupo (métricas)
    n_configs = len(config_order)
    n_metrics = len(metrics)
    
    # Ancho total de un grupo y ancho de cada barra
    group_width = 0.8
    bar_width = group_width / n_metrics
    
    # Posiciones base para cada grupo (configuración)
    group_positions = np.arange(n_configs)
    
    # Dibujar barras para cada métrica
    for i, metric in enumerate(metrics):
        # Calcular posición de las barras para esta métrica
        positions = group_positions + (i - n_metrics/2 + 0.5) * bar_width
        
        # Valores para esta métrica en cada configuración
        values = []
        for config in config_order:
            config_data = fw_data[fw_data['config'] == config]
            if len(config_data) > 0:
                values.append(config_data[metric].values[0])
            else:
                values.append(0)
        
        # Dibujar barras
        # Use English names for metric labels
        english_metric = metric_translations.get(metric, metric).replace('_', ' ').title()
        
        bars = ax.bar(positions, values, width=bar_width, 
                    label=english_metric,
                    color=metric_colors.get(metric, f"C{i}"))
        
        # Añadir etiquetas solo si SHOW_VALUE_LABELS es True
        if SHOW_VALUE_LABELS:
            for j, bar in enumerate(bars):
                if values[j] > 0.01:  # Solo mostramos etiquetas para valores significativos
                    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                           f"{values[j]:.3f}", ha='center', va='bottom', 
                           fontsize=9, fontweight='bold', rotation=45)
    
    # Configurar gráfico
    ax.set_title(f"{title} - {framework_name}", fontsize=16, fontweight='bold')
    ax.set_xticks(group_positions)
    ax.set_xticklabels(config_order, rotation=30, ha='right')
    ax.set_ylabel("Value", fontweight='bold')
    ax.set_xlabel("Configuration", fontweight='bold')
    ax.grid(axis='both', linestyle='--', alpha=0.7)
    
    # Leyenda
    ax.legend(title="Metric", frameon=True, loc='upper center', 
             bbox_to_anchor=(0.5, -0.15), ncol=len(metrics))
    
    plt.tight_layout()
    plt.subplots_adjust(bottom=0.2)  # Espacio para la leyenda
    
    plt.show()

# Función para crear gráfico de framework con líneas de tendencia polinómicas y R² en leyenda
def plot_framework_metrics_with_trend(data, framework_name, metrics, title):
    """
    Creates a chart showing the metrics of a specific framework across all configurations,
    with polynomial trend lines and R² values included in the legend.
    
    Args:
        data: DataFrame with the data
        framework_name: Name of the framework to analyze
        metrics: List of metrics to show
        title: Chart title
    """
    # Filtrar datos solo para el framework especificado
    fw_data = data[data['framework'] == framework_name].copy()
    
    # Preparar figura
    fig, ax = plt.subplots(figsize=(14, 7))
    
    # Extraemos los índices de configuración
    config_indices = list(range(len(config_order)))
    
    # Almacenar los valores R² para incluirlos en la leyenda
    r_squared_values = {}
    
    # Dibujar líneas para cada métrica
    for i, metric in enumerate(metrics):
        # Preparamos arrays para valores e índices
        metric_values = []
        metric_indices = []
        
        # Para cada configuración, extraemos el valor si existe
        for j, config in enumerate(config_order):
            config_data = fw_data[fw_data['config'] == config]
            if len(config_data) > 0:
                val = config_data[metric].values[0]
                # Verificamos que el valor sea válido
                if not np.isnan(val):
                    metric_values.append(val)
                    metric_indices.append(j)
        
        # Si no hay suficientes datos, continuamos con la siguiente métrica
        if len(metric_indices) < 3:
            continue
        
        # Use English names for metric labels
        english_metric = metric_translations.get(metric, metric).replace('_', ' ').title()
        
        # Dibujar línea con marcadores
        line, = ax.plot(metric_indices, metric_values, 'o-', 
                      label=english_metric,
                      color=metric_colors.get(metric, f"C{i}"),
                      linewidth=2, markersize=8)
        
        # Añadir etiquetas de valores solo si SHOW_VALUE_LABELS es True
        if SHOW_VALUE_LABELS:
            for j, val in zip(metric_indices, metric_values):
                if val > 0.01:  # Mostrar solo valores significativos
                    ax.text(j, val * 1.01, 
                           f"{val:.3f}", ha='center', va='bottom',
                           color=metric_colors.get(metric, f"C{i}"),
                           fontsize=9, fontweight='bold', rotation=45)
        
        # Usamos un ajuste polinómico (grado 2) para capturar cambios no lineales
        if len(metric_indices) >= 3:
            z = np.polyfit(metric_indices, metric_values, 2)
            p = np.poly1d(z)
            
            # Generamos puntos adicionales para suavizar la curva
            x_smooth = np.linspace(min(metric_indices), max(metric_indices), 100)
            y_smooth = p(x_smooth)
            
            # Dibujamos la línea de tendencia suavizada
            ax.plot(x_smooth, y_smooth, '--', 
                   color=metric_colors.get(metric, f"C{i}"), 
                   linewidth=1.5, alpha=0.7)
            
            # Calculamos R² para evaluar la calidad del ajuste
            y_pred = p(metric_indices)
            ss_total = np.sum((metric_values - np.mean(metric_values))**2)
            ss_residual = np.sum((metric_values - y_pred)**2)
            r_squared = 1 - (ss_residual / ss_total)
            r_squared_values[english_metric] = r_squared
        else:
            r_squared_values[english_metric] = float('nan')
    
    # Configurar gráfico
    ax.set_title(f"{title} - {framework_name} with Polynomial Trend Lines", fontsize=16, fontweight='bold')
    ax.set_xticks(config_indices)
    ax.set_xticklabels(config_order, rotation=30, ha='right')
    ax.set_ylabel("Value", fontweight='bold')
    ax.set_xlabel("Configuration", fontweight='bold')
    ax.grid(axis='both', linestyle='--', alpha=0.7)
    
    # Añadir áreas sombreadas para separar tamaños de población
    pop_sizes = ["2^6", "2^10", "2^14"]
    colors = ["#f0f0f0", "#e8e8e8", "#f0f0f0"]
    
    for p_idx, pop in enumerate(pop_sizes):
        pop_configs = [j for j, c in enumerate(config_order) if c.startswith(pop)]
        if pop_configs:
            start, end = min(pop_configs), max(pop_configs)
            ax.axvspan(start - 0.5, end + 0.5, alpha=0.2, color=colors[p_idx])
            ax.text((start + end) / 2, 0.98, 
                   f"Population {pop}", ha='center', fontsize=10, alpha=0.7,
                   transform=ax.get_xaxis_transform())
    
    # Crear etiquetas personalizadas para la leyenda que incluyan R²
    custom_labels = []
    custom_handles = []
    
    # Obtenemos los handles actuales y sus etiquetas
    handles, labels = ax.get_legend_handles_labels()
    
    for handle, label in zip(handles, labels):
        if label in r_squared_values:
            r2 = r_squared_values[label]
            if not np.isnan(r2):
                # Incluimos el valor R² en la etiqueta
                custom_labels.append(f"{label} (R²={r2:.2f})")
            else:
                custom_labels.append(label)
            custom_handles.append(handle)
    
    # Crear la leyenda con las etiquetas personalizadas
    ax.legend(custom_handles, custom_labels, title="Metric", 
             frameon=True, loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=len(metrics))
    
    plt.tight_layout()
    plt.subplots_adjust(bottom=0.2)  # Espacio para la leyenda
    
    plt.show()
# Para usar el código, simplemente cambia el valor de SHOW_VALUE_LABELS
# SHOW_VALUE_LABELS = True   # Para mostrar los números en las gráficas
# SHOW_VALUE_LABELS = False  # Para ocultar los números en las gráficas

# Métricas de fitness a comparar en cada gráfico
fitness_metrics = ["fitness_inicial", "variacion_fitness", "fitness_maximo"]

# Generamos gráficos agrupados por tamaño de población
plot_by_population_size(df_resultados, "2^6", fitness_metrics, "Fitness Metrics Comparison - Laptop")
plot_by_population_size(df_resultados, "2^10", fitness_metrics, "Fitness Metrics Comparison - Laptop")
plot_by_population_size(df_resultados, "2^14", fitness_metrics, "Fitness Metrics Comparison - Laptop")

# Generar gráficos por framework (versión original con barras y nueva con líneas de tendencia)
for fw in df_resultados['framework'].unique():
    plot_framework_metrics(df_resultados, fw, fitness_metrics, "Fitness Metrics Comparison - Laptop")
    plot_framework_metrics_with_trend(df_resultados, fw, fitness_metrics, "Fitness Metrics Comparison - Laptop")


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats

# Add a global control variable to show or hide value labels
SHOW_VALUE_LABELS = False  # Set to False to hide all value labels on charts

# Configuración de estilo para gráficos más atractivos
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = [14, 8]
plt.rcParams['font.size'] = 12
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['legend.fontsize'] = 12
plt.rcParams['axes.titlepad'] = 20
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False

# English translations for metric names
metric_translations = {
    "fitness_inicial": "initial_fitness",
    "variacion_fitness": "fitness_variation",
    "fitness_maximo": "maximum_fitness",
    "generacion": "generation",
    "tiempo_transcurrido": "elapsed_time"
}

# Archivos de resultados
files_resultados_pablo = {
    "ECJ": "resultados_ECJ_pablo.xlsx",
    "Inspyred": "resultados_inspyred_pablo_3.xlsx",
    "DEAP": "resultados_DEAP_pablo.xlsx",
    "Paradiseo": "resultados_paradiseo_pablo.xlsx"
}
# Columnas de interés para comparar resultados
cols_resultados = [
    "fitness_inicial",
    "variacion_fitness",
    "fitness_maximo",  
    "generacion",
    "tiempo_transcurrido"
]

# Mapa manual de configuraciones
config_map = {
    0: "2^6, 0.01",
    1: "2^6, 0.2",
    2: "2^6, 0.8",
    3: "2^10, 0.01",
    4: "2^10, 0.2",
    5: "2^10, 0.8",
    6: "2^14, 0.01",
    7: "2^14, 0.2",
    8: "2^14, 0.8",
}

# Orden correcto de configuraciones
config_order = [config_map[i] for i in range(9)]

# Colores personalizados para cada framework
framework_colors = {
    "ECJ": "#1f77b4",       # Azul
    "Inspyred": "#ff7f0e",  # Naranja
    "DEAP": "#2ca02c",      # Verde
    "Paradiseo": "#d62728"  # Rojo
}

# Colores para las diferentes métricas
metric_colors = {
    "fitness_inicial": "#3498db",    # Azul claro
    "variacion_fitness": "#2ecc71",  # Verde claro
    "fitness_maximo": "#e74c3c"      # Rojo claro
}

# Leemos y unificamos todos los datos
df_list = []
for fw_name, file_path in files_resultados_pablo.items():
    temp = pd.read_excel(file_path, sheet_name="MEDIAS")
    
    # Filtramos solo columnas relevantes si existen
    existing_cols = [c for c in cols_resultados if c in temp.columns]
    df_temp = temp[existing_cols].copy()
    
    # Añadimos el nombre del framework
    df_temp["framework"] = fw_name
    
    # Asignamos configuración usando el índice
    df_temp.reset_index(drop=True, inplace=True)
    df_temp["config"] = df_temp.index.map(config_map)
    
    df_list.append(df_temp)

# Concatenamos en un solo DataFrame
df_resultados = pd.concat(df_list, ignore_index=True).fillna(0)

# Función para crear gráficas de barras agrupadas con múltiples métricas
def plot_metrics_comparison(data, config_values, frameworks, metrics, title):
    """
    Creates a chart showing multiple metrics for each framework in a specific configuration.
    
    Args:
        data: DataFrame with the data
        config_values: List of configuration values to include
        frameworks: List of frameworks to compare
        metrics: List of metrics to show for each framework
        title: Chart title
    """
    # Filtrar datos para las configuraciones específicas
    filtered_data = data[data['config'].isin(config_values)].copy()
    
    # Creamos una figura con subplots para cada configuración
    n_configs = len(config_values)
    fig, axes = plt.subplots(1, n_configs, figsize=(18, 6), sharey=True)
    
    # Si solo hay una configuración, axes no será un array, lo convertimos
    if n_configs == 1:
        axes = [axes]
    
    # Para cada configuración
    for i, config in enumerate(config_values):
        ax = axes[i]
        config_data = filtered_data[filtered_data['config'] == config]
        
        # Para cada framework, creamos un grupo de barras (una por métrica)
        bar_width = 0.8 / len(frameworks)
        
        for j, framework in enumerate(frameworks):
            fw_data = config_data[config_data['framework'] == framework]
            if len(fw_data) == 0:
                continue
                
            x_positions = np.arange(len(metrics))
            values = [fw_data[metric].values[0] if not fw_data[metric].empty else 0 for metric in metrics]
            
            # Posición de las barras de este framework
            x_offset = j * bar_width - (len(frameworks) - 1) * bar_width / 2
            
            # Dibujamos las barras para este framework
            bars = ax.bar(x_positions + x_offset, values, width=bar_width, 
                         label=framework if i == 0 else "", 
                         color=framework_colors.get(framework))
            
            # Añadimos etiquetas de valores para barras significativas solo si SHOW_VALUE_LABELS es True
            if SHOW_VALUE_LABELS:
                for k, bar in enumerate(bars):
                    if values[k] > 0.01:  # Solo mostramos etiquetas para valores significativos
                        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                               f"{values[k]:.3f}", ha='center', va='bottom', 
                               fontsize=8, fontweight='bold', rotation=45)
        
        # Configuramos cada subplot
        ax.set_title(f"Configuration: {config}", fontsize=12, fontweight='bold')
        ax.set_xticks(range(len(metrics)))
        
        # Use English names for metrics
        english_metrics = [metric_translations.get(m, m).replace('_', ' ').title() for m in metrics]
        ax.set_xticklabels(english_metrics, rotation=45, ha='right')
        ax.grid(axis='both', linestyle='--', alpha=0.7)
        
        # Para el primer subplot, añadimos etiqueta del eje Y
        if i == 0:
            ax.set_ylabel("Value", fontweight='bold')
    
    # Título general
    fig.suptitle(title, fontsize=16, fontweight='bold', y=1.05)
    
    # Leyenda única para toda la figura
    handles, labels = axes[0].get_legend_handles_labels()
    fig.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, 0), 
              ncol=len(frameworks), frameon=True, fancybox=True, shadow=True)
    
    plt.tight_layout()
    plt.subplots_adjust(bottom=0.2)  # Espacio para la leyenda
    
    plt.show()

# Función para crear gráficas por tamaño de población
def plot_by_population_size(data, pop_size, metrics, title):
    """
    Creates charts for configurations with a specific population size.
    
    Args:
        data: DataFrame with the data
        pop_size: Population size ("2^6", "2^10", "2^14")
        metrics: List of metrics to show
        title: Base title for the chart
    """
    # Filtramos configuraciones para este tamaño de población
    configs = [c for c in config_order if c.startswith(pop_size)]
    
    # Frameworks disponibles
    frameworks = data['framework'].unique()
    
    plot_metrics_comparison(
        data=data,
        config_values=configs,
        frameworks=frameworks,
        metrics=metrics,
        title=f"{title} (Population {pop_size})"
    )

# Métricas de fitness a comparar en cada gráfico
fitness_metrics = ["fitness_inicial", "variacion_fitness", "fitness_maximo"]

# Generamos gráficos agrupados por tamaño de población
plot_by_population_size(df_resultados, "2^6", fitness_metrics, "Fitness Metrics Comparison - Server")
plot_by_population_size(df_resultados, "2^10", fitness_metrics, "Fitness Metrics Comparison - Server")
plot_by_population_size(df_resultados, "2^14", fitness_metrics, "Fitness Metrics Comparison - Server")

# Función para crear gráfico de framework con líneas de tendencia polinómicas y R² en leyenda
def plot_framework_metrics_with_trend(data, framework_name, metrics, title):
    """
    Creates a chart showing the metrics of a specific framework across all configurations,
    with polynomial trend lines and R² values included in the legend.
    
    Args:
        data: DataFrame with the data
        framework_name: Name of the framework to analyze
        metrics: List of metrics to show
        title: Chart title
    """
    # Filtrar datos solo para el framework especificado
    fw_data = data[data['framework'] == framework_name].copy()
    
    # Preparar figura
    fig, ax = plt.subplots(figsize=(14, 7))
    
    # Extraemos los índices de configuración
    config_indices = list(range(len(config_order)))
    
    # Almacenar los valores R² para incluirlos en la leyenda
    r_squared_values = {}
    
    # Dibujar líneas para cada métrica
    for i, metric in enumerate(metrics):
        # Preparamos arrays para valores e índices
        metric_values = []
        metric_indices = []
        
        # Para cada configuración, extraemos el valor si existe
        for j, config in enumerate(config_order):
            config_data = fw_data[fw_data['config'] == config]
            if len(config_data) > 0:
                val = config_data[metric].values[0]
                # Verificamos que el valor sea válido
                if not np.isnan(val):
                    metric_values.append(val)
                    metric_indices.append(j)
        
        # Si no hay suficientes datos, continuamos con la siguiente métrica
        if len(metric_indices) < 3:
            continue
        
        # Use English names for metric labels
        english_metric = metric_translations.get(metric, metric).replace('_', ' ').title()
        
        # Dibujar línea con marcadores
        line, = ax.plot(metric_indices, metric_values, 'o-', 
                      label=english_metric,
                      color=metric_colors.get(metric, f"C{i}"),
                      linewidth=2, markersize=8)
        
        # Añadir etiquetas de valores solo si SHOW_VALUE_LABELS es True
        if SHOW_VALUE_LABELS:
            for j, val in zip(metric_indices, metric_values):
                if val > 0.01:  # Mostrar solo valores significativos
                    ax.text(j, val * 1.01, 
                           f"{val:.3f}", ha='center', va='bottom',
                           color=metric_colors.get(metric, f"C{i}"),
                           fontsize=9, fontweight='bold', rotation=45)
        
        # Usamos un ajuste polinómico (grado 2) para capturar cambios no lineales
        if len(metric_indices) >= 3:
            z = np.polyfit(metric_indices, metric_values, 2)
            p = np.poly1d(z)
            
            # Generamos puntos adicionales para suavizar la curva
            x_smooth = np.linspace(min(metric_indices), max(metric_indices), 100)
            y_smooth = p(x_smooth)
            
            # Dibujamos la línea de tendencia suavizada
            ax.plot(x_smooth, y_smooth, '--', 
                   color=metric_colors.get(metric, f"C{i}"), 
                   linewidth=1.5, alpha=0.7)
            
            # Calculamos R² para evaluar la calidad del ajuste
            y_pred = p(metric_indices)
            ss_total = np.sum((metric_values - np.mean(metric_values))**2)
            ss_residual = np.sum((metric_values - y_pred)**2)
            r_squared = 1 - (ss_residual / ss_total)
            r_squared_values[english_metric] = r_squared
        else:
            r_squared_values[english_metric] = float('nan')
    
    # Configurar gráfico
    ax.set_title(f"{title} - {framework_name} with Polynomial Trend Lines", fontsize=16, fontweight='bold')
    ax.set_xticks(config_indices)
    ax.set_xticklabels(config_order, rotation=30, ha='right')
    ax.set_ylabel("Value", fontweight='bold')
    ax.set_xlabel("Configuration", fontweight='bold')
    ax.grid(axis='both', linestyle='--', alpha=0.7)
    
    # Añadir áreas sombreadas para separar tamaños de población
    pop_sizes = ["2^6", "2^10", "2^14"]
    colors = ["#f0f0f0", "#e8e8e8", "#f0f0f0"]
    
    for p_idx, pop in enumerate(pop_sizes):
        pop_configs = [j for j, c in enumerate(config_order) if c.startswith(pop)]
        if pop_configs:
            start, end = min(pop_configs), max(pop_configs)
            ax.axvspan(start - 0.5, end + 0.5, alpha=0.2, color=colors[p_idx])
            ax.text((start + end) / 2, 0.98, 
                   f"Population {pop}", ha='center', fontsize=10, alpha=0.7,
                   transform=ax.get_xaxis_transform())
    
    # Crear etiquetas personalizadas para la leyenda que incluyan R²
    custom_labels = []
    custom_handles = []
    
    # Obtenemos los handles actuales y sus etiquetas
    handles, labels = ax.get_legend_handles_labels()
    
    for handle, label in zip(handles, labels):
        if label in r_squared_values:
            r2 = r_squared_values[label]
            if not np.isnan(r2):
                # Incluimos el valor R² en la etiqueta
                custom_labels.append(f"{label} (R²={r2:.2f})")
            else:
                custom_labels.append(label)
            custom_handles.append(handle)
    
    # Crear la leyenda con las etiquetas personalizadas
    ax.legend(custom_handles, custom_labels, title="Metric", 
             frameon=True, loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=len(metrics))
    
    plt.tight_layout()
    plt.subplots_adjust(bottom=0.2)  # Espacio para la leyenda
    
    plt.show()
    
    # También crear un gráfico de barras tradicional para comparación
    plot_framework_metrics(data, framework_name, metrics, title)
    
# Versión original de gráfico de barras para comparación
def plot_framework_metrics(data, framework_name, metrics, title):
    """
    Creates a chart showing the metrics of a specific framework across all configurations.
    
    Args:
        data: DataFrame with the data
        framework_name: Name of the framework to analyze
        metrics: List of metrics to show
        title: Chart title
    """
    # Filtrar datos solo para el framework especificado
    fw_data = data[data['framework'] == framework_name].copy()
    
    # Preparar figura
    fig, ax = plt.subplots(figsize=(14, 7))
    
    # Número de grupos (configuraciones) y de barras por grupo (métricas)
    n_configs = len(config_order)
    n_metrics = len(metrics)
    
    # Ancho total de un grupo y ancho de cada barra
    group_width = 0.8
    bar_width = group_width / n_metrics
    
    # Posiciones base para cada grupo (configuración)
    group_positions = np.arange(n_configs)
    
    # Dibujar barras para cada métrica
    for i, metric in enumerate(metrics):
        # Calcular posición de las barras para esta métrica
        positions = group_positions + (i - n_metrics/2 + 0.5) * bar_width
        
        # Valores para esta métrica en cada configuración
        values = []
        for config in config_order:
            config_data = fw_data[fw_data['config'] == config]
            if len(config_data) > 0:
                values.append(config_data[metric].values[0])
            else:
                values.append(0)
        
        # Dibujar barras
        # Use English names for metric labels
        english_metric = metric_translations.get(metric, metric).replace('_', ' ').title()
        
        bars = ax.bar(positions, values, width=bar_width, 
                    label=english_metric,
                    color=metric_colors.get(metric, f"C{i}"))
        
        # Añadir etiquetas solo si SHOW_VALUE_LABELS es True
        if SHOW_VALUE_LABELS:
            for j, bar in enumerate(bars):
                if values[j] > 0.01:  # Solo mostramos etiquetas para valores significativos
                    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                           f"{values[j]:.3f}", ha='center', va='bottom', 
                           fontsize=9, fontweight='bold', rotation=45)
    
    # Configurar gráfico
    ax.set_title(f"{title} - {framework_name}", fontsize=16, fontweight='bold')
    ax.set_xticks(group_positions)
    ax.set_xticklabels(config_order, rotation=30, ha='right')
    ax.set_ylabel("Value", fontweight='bold')
    ax.set_xlabel("Configuration", fontweight='bold')
    ax.grid(axis='both', linestyle='--', alpha=0.7)
    
    # Leyenda
    ax.legend(title="Metric", frameon=True, loc='upper center', 
             bbox_to_anchor=(0.5, -0.15), ncol=len(metrics))
    
    plt.tight_layout()
    plt.subplots_adjust(bottom=0.2)  # Espacio para la leyenda
    
    plt.show()

# Generar gráficos por framework con líneas de tendencia
for fw in df_resultados['framework'].unique():
    plot_framework_metrics_with_trend(df_resultados, fw, fitness_metrics, "Fitness Metrics Comparison - Server")

ESTADÍSTICAS RESULTADOS EN EL SERVIDOR

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats

# Variable global para controlar si se muestran los valores numéricos en las gráficas
SHOW_VALUE_LABELS = True  # Cambiar a False para ocultar las etiquetas de valores

# Configuración de estilo para gráficos más atractivos
plt.style.use('seaborn-v0_8-whitegrid')     # Estilo base de seaborn
plt.rcParams['figure.figsize'] = (14, 7)    # Tamaño más grande
plt.rcParams['font.size'] = 12              # Texto más grande
plt.rcParams['axes.labelsize'] = 14         # Etiquetas más grandes
plt.rcParams['axes.titlesize'] = 16         # Título más grande
plt.rcParams['xtick.labelsize'] = 12        # Etiquetas del eje X más grandes
plt.rcParams['ytick.labelsize'] = 12        # Etiquetas del eje Y más grandes
plt.rcParams['legend.fontsize'] = 12        # Leyenda más grande
plt.rcParams['axes.titlepad'] = 20          # Espacio para el título
plt.rcParams['axes.spines.top'] = False     # Eliminar línea superior
plt.rcParams['axes.spines.right'] = False   # Eliminar línea derecha

# Archivos de resultados del portátil de Pablo
files_resultados_servidor = {
    "ECJ": "resultados_ECJ_pablo.xlsx",
    "Inspyred": "resultados_inspyred_pablo_3.xlsx",
    "DEAP": "resultados_DEAP_pablo.xlsx",
    "Paradiseo": "resultados_paradiseo_pablo.xlsx"
}

# Columnas de interés para comparar resultados
cols_resultados = [
    "generacion", 
    "fitness_inicial",
    "variacion_fitness",
    "fitness_maximo",
    "tiempo_transcurrido"
]

# Traducción de métricas al inglés
metric_translations = {
    "fitness_inicial": "Initial Fitness",
    "variacion_fitness": "Fitness Variation",
    "fitness_maximo": "Maximum Fitness",
    "generacion": "Generation",
    "tiempo_transcurrido": "Elapsed Time"
}

# Mapa manual de configuraciones
config_map = {
    0: "2^6, 0.01",
    1: "2^6, 0.2",
    2: "2^6, 0.8",
    3: "2^10, 0.01",
    4: "2^10, 0.2",
    5: "2^10, 0.8",
    6: "2^14, 0.01",
    7: "2^14, 0.2",
    8: "2^14, 0.8",
}

# Orden correcto de configuraciones para usar en gráficos
config_order = [config_map[i] for i in range(9)]

# Colores personalizados para cada framework
framework_colors = {
    "ECJ": "#1f77b4",       # Azul
    "Inspyred": "#ff7f0e",  # Naranja
    "DEAP": "#2ca02c",      # Verde
    "Paradiseo": "#d62728"  # Rojo
}

# Colores para las diferentes métricas
metric_colors = {
    "fitness_inicial": "#3498db",    # Azul claro
    "variacion_fitness": "#2ecc71",  # Verde claro
    "fitness_maximo": "#e74c3c",     # Rojo claro
    "generacion": "#9b59b6",         # Púrpura
    "tiempo_transcurrido": "#f39c12" # Naranja
}

# Leemos y unificamos todos los datos
df_list = []
for fw_name, file_path in files_resultados_servidor.items():
    temp = pd.read_excel(file_path, sheet_name="MEDIAS")
    
    # Filtramos solo columnas relevantes si existen
    existing_cols = [c for c in cols_resultados if c in temp.columns]
    df_temp = temp[existing_cols].copy()
    
    # Añadimos el nombre del framework
    df_temp["framework"] = fw_name
    
    # Asignamos configuración usando el índice
    df_temp.reset_index(drop=True, inplace=True)
    df_temp["config"] = df_temp.index.map(config_map)
    
    df_list.append(df_temp)

# Concatenamos en un solo DataFrame
df_resultados_pablo = pd.concat(df_list, ignore_index=True).fillna(0)

# ───────────────────────────────────────────────────────────
# Función mejorada para hacer gráficas agrupadas
# ───────────────────────────────────────────────────────────
def plot_grouped_bars(data, x_col, group_col, y_col, title, ylabel, y_min_pct=0.9, log_scale=False):
    """
    Crea un gráfico de barras agrupadas de la columna 'y_col',
    con 'x_col' en el eje X y cada valor de 'group_col' como una barra.
    
    Args:
        data: DataFrame con los datos
        x_col: Columna para el eje X (categorías)
        group_col: Columna para agrupar (diferentes barras)
        y_col: Columna con los valores a mostrar
        title: Título del gráfico
        ylabel: Etiqueta del eje Y
        y_min_pct: Porcentaje del valor mínimo para ajustar el eje Y (0-1)
        log_scale: Si es True, usa escala logarítmica en el eje Y
    """
    # 1) Pasamos a formato ancho: filas = config, columnas = framework, valores = y_col
    pivoted = data.pivot(index=x_col, columns=group_col, values=y_col)
    
    # Reordenamos las filas según el orden definido en config_order
    pivoted = pivoted.reindex(config_order)
    
    # Obtenemos los valores mínimos y máximos para ajustar el eje Y
    y_min = pivoted.values.min() * y_min_pct
    y_max = pivoted.values.max() * 1.15  # 15% de margen superior
    
    # Creamos el gráfico con más espacio a la derecha para la leyenda
    fig, ax = plt.subplots(figsize=(14, 7))  # Figura más ancha para acomodar la leyenda
    pivoted.plot(
        kind="bar", 
        ax=ax,
        color=[framework_colors.get(fw, "#333333") for fw in pivoted.columns],
        width=0.7
    )
    
    # Configuración de escala y límites
    if log_scale:
        plt.yscale('log')  # Escala logarítmica para el eje Y
        # Ajustamos el límite superior para evitar que las etiquetas se salgan
        plt.ylim(top=pivoted.values.max() * 1.3)  # 30% de margen superior
    else:
        plt.ylim(y_min, y_max)
    
    # Título y etiquetas con estilo mejorado
    plt.title(title, fontweight='bold', pad=20)
    plt.xlabel("Configuration", fontweight='bold')
    plt.ylabel(ylabel, fontweight='bold')
    plt.xticks(rotation=30, ha="right")
    
    # Destacamos la barra mayor en cada fila (configuración)
    for row_idx, config_name in enumerate(pivoted.index):
        # Obtenemos la fila con sus frameworks y valores
        row_values = pivoted.loc[config_name]
        # Nombre de la columna (framework) con el valor máximo en esta fila
        max_col = row_values.idxmax()
        # Valor máximo
        max_value = row_values[max_col]
        # Índice que corresponde a ese framework en pivoted.columns
        col_idx = list(pivoted.columns).index(max_col)
        
        # Localizamos la barra que corresponde al row_idx en ese contenedor
        bar = ax.containers[col_idx][row_idx]
        
        # Añadimos etiqueta solo a la barra con el valor máximo - controlado por SHOW_VALUE_LABELS
        if SHOW_VALUE_LABELS:
            if log_scale:
                label_y_pos = max_value * 1.05  # Más cerca en escala logarítmica
            else:
                label_y_pos = max_value * 1.01  # Ligeramente por encima en escala normal
                
            ax.text(
                bar.get_x() + bar.get_width()/2,
                label_y_pos,
                f"{max_value:.4g}",
                ha='center',
                va='bottom',
                fontweight='bold',
                color='black',
                fontsize=11
            )
    
    # Mejoramos la leyenda y la colocamos a la derecha del gráfico
    legend = plt.legend(title="Framework", frameon=True, loc='center left', bbox_to_anchor=(1.0, 0.5))
    legend.get_frame().set_facecolor('#F8F8F8')
    legend.get_frame().set_edgecolor('#CCCCCC')
    
    # Añadimos grid solo en el eje Y para facilitar la lectura
    plt.grid(axis='both', linestyle='--', alpha=0.7)
    
    # Ajustamos el layout teniendo en cuenta el espacio para la leyenda
    plt.tight_layout(rect=[0, 0, 0.85, 1])
    
    plt.show()

# ───────────────────────────────────────────────────────────
# Función para crear gráficas de líneas con líneas de tendencia y R²
# ───────────────────────────────────────────────────────────
# Función para crear gráfico de framework con líneas de tendencia polinómicas y R² en leyenda
def plot_framework_metrics_with_trend(data, framework_name, metrics, title):
    """
    Creates a chart showing the metrics of a specific framework across all configurations,
    with polynomial trend lines and R² values included in the legend.
    
    Args:
        data: DataFrame with the data
        framework_name: Name of the framework to analyze
        metrics: List of metrics to show
        title: Chart title
    """
    # Filtrar datos solo para el framework especificado
    fw_data = data[data['framework'] == framework_name].copy()
    
    # Preparar figura
    fig, ax = plt.subplots(figsize=(14, 7))
    
    # Extraemos los índices de configuración
    config_indices = list(range(len(config_order)))
    
    # Almacenar los valores R² para incluirlos en la leyenda
    r_squared_values = {}
    
    # Dibujar líneas para cada métrica
    for i, metric in enumerate(metrics):
        # Preparamos arrays para valores e índices
        metric_values = []
        metric_indices = []
        
        # Para cada configuración, extraemos el valor si existe
        for j, config in enumerate(config_order):
            config_data = fw_data[fw_data['config'] == config]
            if len(config_data) > 0:
                val = config_data[metric].values[0]
                # Verificamos que el valor sea válido
                if not np.isnan(val):
                    metric_values.append(val)
                    metric_indices.append(j)
        
        # Si no hay suficientes datos, continuamos con la siguiente métrica
        if len(metric_indices) < 3:
            continue
        
        # Use English names for metric labels
        english_metric = metric_translations.get(metric, metric).replace('_', ' ').title()
        
        # Dibujar línea con marcadores
        line, = ax.plot(metric_indices, metric_values, 'o-', 
                      label=english_metric,
                      color=metric_colors.get(metric, f"C{i}"),
                      linewidth=2, markersize=8)
        
        # Añadir etiquetas de valores solo si SHOW_VALUE_LABELS es True
        if SHOW_VALUE_LABELS:
            for j, val in zip(metric_indices, metric_values):
                if val > 0.01:  # Mostrar solo valores significativos
                    ax.text(j, val * 1.01, 
                           f"{val:.3f}", ha='center', va='bottom',
                           color=metric_colors.get(metric, f"C{i}"),
                           fontsize=9, fontweight='bold', rotation=45)
        
        # Usamos un ajuste polinómico (grado 2) para capturar cambios no lineales
        if len(metric_indices) >= 3:
            z = np.polyfit(metric_indices, metric_values, 2)
            p = np.poly1d(z)
            
            # Generamos puntos adicionales para suavizar la curva
            x_smooth = np.linspace(min(metric_indices), max(metric_indices), 100)
            y_smooth = p(x_smooth)
            
            # Dibujamos la línea de tendencia suavizada
            ax.plot(x_smooth, y_smooth, '--', 
                   color=metric_colors.get(metric, f"C{i}"), 
                   linewidth=1.5, alpha=0.7)
            
            # Calculamos R² para evaluar la calidad del ajuste
            y_pred = p(metric_indices)
            ss_total = np.sum((metric_values - np.mean(metric_values))**2)
            ss_residual = np.sum((metric_values - y_pred)**2)
            r_squared = 1 - (ss_residual / ss_total)
            r_squared_values[english_metric] = r_squared
        else:
            r_squared_values[english_metric] = float('nan')
    
    # Configurar gráfico
    ax.set_title(f"{title} - {framework_name} with Polynomial Trend Lines", fontsize=16, fontweight='bold')
    ax.set_xticks(config_indices)
    ax.set_xticklabels(config_order, rotation=30, ha='right')
    ax.set_ylabel("Value", fontweight='bold')
    ax.set_xlabel("Configuration", fontweight='bold')
    ax.grid(axis='both', linestyle='--', alpha=0.7)
    
    # Añadir áreas sombreadas para separar tamaños de población
    pop_sizes = ["2^6", "2^10", "2^14"]
    colors = ["#f0f0f0", "#e8e8e8", "#f0f0f0"]
    
    for p_idx, pop in enumerate(pop_sizes):
        pop_configs = [j for j, c in enumerate(config_order) if c.startswith(pop)]
        if pop_configs:
            start, end = min(pop_configs), max(pop_configs)
            ax.axvspan(start - 0.5, end + 0.5, alpha=0.2, color=colors[p_idx])
            ax.text((start + end) / 2, 0.98, 
                   f"Population {pop}", ha='center', fontsize=10, alpha=0.7,
                   transform=ax.get_xaxis_transform())
    
    # Crear etiquetas personalizadas para la leyenda que incluyan R²
    custom_labels = []
    custom_handles = []
    
    # Obtenemos los handles actuales y sus etiquetas
    handles, labels = ax.get_legend_handles_labels()
    
    for handle, label in zip(handles, labels):
        if label in r_squared_values:
            r2 = r_squared_values[label]
            if not np.isnan(r2):
                # Incluimos el valor R² en la etiqueta
                custom_labels.append(f"{label} (R²={r2:.2f})")
            else:
                custom_labels.append(label)
            custom_handles.append(handle)
    
    # Crear la leyenda con las etiquetas personalizadas
    ax.legend(custom_handles, custom_labels, title="Metric", 
             frameon=True, loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=len(metrics))
    
    plt.tight_layout()
    plt.subplots_adjust(bottom=0.2)  # Espacio para la leyenda
    
    plt.show()

# Para usar el código, simplemente cambia el valor de SHOW_VALUE_LABELS
# SHOW_VALUE_LABELS = True   # Para mostrar los números en las gráficas
# SHOW_VALUE_LABELS = False  # Para ocultar los números en las gráficas

# ───────────────────────────────────────────────────────────
# Hacemos una gráfica para cada métrica de resultados
# ───────────────────────────────────────────────────────────
# 1. Gráficos de barras para cada métrica
if "generacion" in df_resultados_pablo.columns:
    plot_grouped_bars(
        data=df_resultados_pablo,
        x_col="config",
        group_col="framework",
        y_col="generacion",
        title="Completed Generations: Framework Comparison (Server)",
        ylabel="Generations",
        y_min_pct=0.85,
        log_scale=True  # Usamos escala logarítmica para generaciones (pueden tener valores muy diferentes)
    )

if "fitness_inicial" in df_resultados_pablo.columns:
    plot_grouped_bars(
        data=df_resultados_pablo,
        x_col="config",
        group_col="framework",
        y_col="fitness_inicial",
        title="Initial Fitness: Framework Comparison (Server)",
        ylabel="Initial Fitness",
        y_min_pct=0.85
    )

if "variacion_fitness" in df_resultados_pablo.columns:
    plot_grouped_bars(
        data=df_resultados_pablo,
        x_col="config",
        group_col="framework",
        y_col="variacion_fitness",
        title="Fitness Variation: Framework Comparison (Server)",
        ylabel="Fitness Variation",
        y_min_pct=0.85
    )

if "fitness_maximo" in df_resultados_pablo.columns:
    plot_grouped_bars(
        data=df_resultados_pablo,
        x_col="config",
        group_col="framework",
        y_col="fitness_maximo",
        title="Maximum Fitness: Framework Comparison (Server)",
        ylabel="Maximum Fitness",
        y_min_pct=0.85
    )

if "tiempo_transcurrido" in df_resultados_pablo.columns:
    plot_grouped_bars(
        data=df_resultados_pablo,
        x_col="config",
        group_col="framework",
        y_col="tiempo_transcurrido",
        title="Elapsed Time: Framework Comparison (Server)",
        ylabel="Time (ms)",
        y_min_pct=0.85
    )

# 2. Gráficos con líneas de tendencia para cada framework
fitness_metrics = ["fitness_inicial", "variacion_fitness", "fitness_maximo"]
for fw in df_resultados_pablo['framework'].unique():
    plot_framework_metrics_with_trend(df_resultados_pablo, fw, fitness_metrics, "Fitness Metrics Comparison (Server)")

ahora vamos a ver fitness/consumo

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats

# Variable global para controlar si se muestran los valores numéricos en las gráficas
SHOW_VALUE_LABELS = False  # Set to False to hide all value labels on charts

# Configuración de estilo para gráficos
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (14, 7)
plt.rcParams['font.size'] = 12
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['legend.fontsize'] = 12
plt.rcParams['axes.titlepad'] = 20
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False

# Mapa manual de configuraciones
config_map = {
    0: "2^6, 0.01",
    1: "2^6, 0.2",
    2: "2^6, 0.8",
    3: "2^10, 0.01",
    4: "2^10, 0.2",
    5: "2^10, 0.8",
    6: "2^14, 0.01",
    7: "2^14, 0.2",
    8: "2^14, 0.8",
}

# Orden correcto de configuraciones
config_order = [config_map[i] for i in range(9)]

# Colores personalizados para cada framework
framework_colors = {
    "ECJ": "#1f77b4",       # Azul
    "Inspyred": "#ff7f0e",  # Naranja
    "DEAP": "#2ca02c",      # Verde
    "Paradiseo": "#d62728"  # Rojo
}

# Traducción de términos - CORREGIDO
translations = {
    "ordenador": "Laptop",
    "pablo": "Server",
    "fitness_maximo": "maximum_fitness",
    "variacion_fitness": "fitness_variation",
    "energy_consumed": "energy_consumed",
    "fitness_per_energy": "fitness_per_energy",
    "variacion_fitness_per_energy": "variation_per_energy"
}

# Colores para combinaciones framework-máquina - CORREGIDO
framework_machine_colors = {
    "ECJ (Laptop)": "#1f77b4",      # Azul oscuro
    "ECJ (Server)": "#7ab6e8",          # Azul claro
    "Inspyred (Laptop)": "#ff7f0e",  # Naranja oscuro
    "Inspyred (Server)": "#ffbb78",      # Naranja claro
    "DEAP (Laptop)": "#2ca02c",      # Verde oscuro
    "DEAP (Server)": "#98df8a",          # Verde claro
    "Paradiseo (Laptop)": "#d62728", # Rojo oscuro
    "Paradiseo (Server)": "#ff9896"      # Rojo claro
}

# Colores para máquinas - CORREGIDO
machine_colors = {
    "ordenador": "#3498db",  # Azul
    "pablo": "#e74c3c",      # Rojo
    "Laptop": "#3498db",     # Azul
    "Server": "#e74c3c"      # Rojo
}

# 1. Cargar datos de resultados (fitness_maximo y variacion_fitness)
# ------------------------------------------------
files_resultados = {
    ("ECJ", "ordenador"): "resultados_ECJ_ordenador.xlsx",
    ("ECJ", "pablo"): "resultados_ECJ_pablo.xlsx",
    ("Inspyred", "ordenador"): "resultados_inspyred_ordenador_3.xlsx",
    ("Inspyred", "pablo"): "resultados_inspyred_pablo_3.xlsx",
    ("DEAP", "ordenador"): "resultados_DEAP_ordenador.xlsx",
    ("DEAP", "pablo"): "resultados_DEAP_pablo.xlsx",
    ("Paradiseo", "ordenador"): "resultados_paradiseo_ordenador.xlsx",
    ("Paradiseo", "pablo"): "resultados_paradiseo_pablo.xlsx"
}

# Columnas de interés para resultados
cols_resultados = ["fitness_maximo", "variacion_fitness"]

# Leer datos de resultados
df_results_list = []
for (fw_name, machine), file_path in files_resultados.items():
    try:
        temp = pd.read_excel(file_path, sheet_name="MEDIAS")
        
        # Filtramos solo columnas relevantes si existen
        existing_cols = [c for c in cols_resultados if c in temp.columns]
        if not existing_cols:
            print(f"Warning: No columns of interest found in {file_path}")
            continue
            
        df_temp = temp[existing_cols].copy()
        
        # Añadimos información
        df_temp["framework"] = fw_name
        df_temp["machine"] = machine
        
        # Asignamos configuración usando el índice
        df_temp.reset_index(drop=True, inplace=True)
        df_temp["config"] = df_temp.index.map(config_map)
        
        df_results_list.append(df_temp)
    except Exception as e:
        print(f"Error loading {file_path}: {e}")

# Concatenamos resultados
df_results = pd.concat(df_results_list, ignore_index=True).fillna(0)

# 2. Cargar datos de consumo energético
# --------------------------------------
files_energy = {
    ("ECJ", "ordenador"): "codecarbon_ECJ_ordenador.xlsx",
    ("ECJ", "pablo"): "codecarbon_ECJ_pablo.xlsx",
    ("Inspyred", "ordenador"): "codecarbon_inspyred_ordenador_3.xlsx",
    ("Inspyred", "pablo"): "codecarbon_inspyred_pablo_3.xlsx",
    ("DEAP", "ordenador"): "codecarbon_DEAP_ordenador.xlsx",
    ("DEAP", "pablo"): "codecarbon_DEAP_pablo.xlsx",
    ("Paradiseo", "ordenador"): "codecarbon_paradiseo_ordenador.xlsx",
    ("Paradiseo", "pablo"): "codecarbon_paradiseo_pablo.xlsx"
}

# Columnas de interés para consumo
cols_energy = ["energy_consumed"]

# Leer datos de consumo
df_energy_list = []
for (fw_name, machine), file_path in files_energy.items():
    try:
        temp = pd.read_excel(file_path, sheet_name="MEDIAS")
        
        # Filtramos solo columnas relevantes si existen
        existing_cols = [c for c in cols_energy if c in temp.columns]
        if not existing_cols:
            print(f"Warning: No consumption columns found in {file_path}")
            continue
            
        df_temp = temp[existing_cols].copy()
        
        # Añadimos información
        df_temp["framework"] = fw_name
        df_temp["machine"] = machine
        
        # Asignamos configuración usando el índice
        df_temp.reset_index(drop=True, inplace=True)
        df_temp["config"] = df_temp.index.map(config_map)
        
        df_energy_list.append(df_temp)
    except Exception as e:
        print(f"Error loading {file_path}: {e}")

# Concatenamos datos de consumo
df_energy = pd.concat(df_energy_list, ignore_index=True).fillna(0)

# 3. Combinar datos de fitness y consumo
# --------------------------------------
# Creamos un identificador único para cada combinación de framework, máquina y configuración
df_results['id'] = df_results['framework'] + '_' + df_results['machine'] + '_' + df_results['config']
df_energy['id'] = df_energy['framework'] + '_' + df_energy['machine'] + '_' + df_energy['config']

# Unimos los dataframes
df_combined = pd.merge(df_results, df_energy, on='id', suffixes=('', '_energy'))

# Limpiamos columnas duplicadas
df_combined = df_combined.drop(columns=[c for c in df_combined.columns if c.endswith('_energy') and c != 'energy_consumed'])

# Calculamos las métricas de eficiencia
# Fitness máximo por unidad de energía
df_combined['fitness_per_energy'] = df_combined['fitness_maximo'] / df_combined['energy_consumed']
# Variación de fitness por unidad de energía
df_combined['variacion_fitness_per_energy'] = df_combined['variacion_fitness'] / df_combined['energy_consumed']

# Manejamos posibles divisiones por cero
df_combined.loc[df_combined['energy_consumed'] == 0, 'fitness_per_energy'] = 0
df_combined.loc[df_combined['energy_consumed'] == 0, 'variacion_fitness_per_energy'] = 0

# Traducción de máquinas al inglés - CORREGIDO
df_combined['machine_en'] = df_combined['machine'].map(lambda x: translations.get(x, x))
# Creamos una columna que combine framework y máquina
df_combined['framework_machine'] = df_combined['framework'] + ' (' + df_combined['machine_en'] + ')'

# 4. Función para mostrar barras agrupadas
# -----------------------------------------
def plot_grouped_bars(data, x_col, group_col, y_col, title, ylabel, y_min_pct=0.9, log_scale=False):
    """
    Creates a grouped bar chart
    """
    # Pasamos a formato ancho
    pivoted = data.pivot(index=x_col, columns=group_col, values=y_col)
    
    # Reordenamos las filas
    if x_col == 'config':
        pivoted = pivoted.reindex(config_order)
    
    # Valores mínimos y máximos para ajustar el eje Y
    if pivoted.values.min() > 0:
        y_min = pivoted.values.min() * y_min_pct
    else:
        y_min = 0
    y_max = pivoted.values.max() * 1.15
    
    # Creamos el gráfico
    fig, ax = plt.subplots(figsize=(14, 7))
    
    # Definimos colores según el grupo
    if group_col == 'framework_machine':
        # Usamos el mapeo específico para combinaciones framework-máquina en inglés
        colors = [framework_machine_colors.get(col, "#333333") for col in pivoted.columns]
    elif group_col == 'framework':
        colors = [framework_colors.get(fw, "#333333") for fw in pivoted.columns]
    elif group_col in ['machine', 'machine_en']:
        colors = [machine_colors.get(m, "#333333") for m in pivoted.columns]
    else:
        # Colores por defecto
        colors = None
    
    pivoted.plot(
        kind="bar", 
        ax=ax,
        color=colors,
        width=0.7
    )
    
    # Configuración de escala y límites
    if log_scale and pivoted.values.min() > 0:
        plt.yscale('log')
        plt.ylim(top=pivoted.values.max() * 1.3)
    else:
        plt.ylim(y_min, y_max)
    
    # Título y etiquetas
    plt.title(title, fontweight='bold', pad=20)
    plt.xlabel("Configuration" if x_col == 'config' else x_col.replace('_', ' ').capitalize(), fontweight='bold')
    plt.ylabel(ylabel, fontweight='bold')
    plt.xticks(rotation=30, ha="right")
    
    # Destacamos la barra mayor en cada fila
    for row_idx, config_name in enumerate(pivoted.index):
        row_values = pivoted.loc[config_name]
        max_col = row_values.idxmax()
        max_value = row_values[max_col]
        col_idx = list(pivoted.columns).index(max_col)
        
        bar = ax.containers[col_idx][row_idx]
        
        # Añadimos etiqueta solo a la barra con el valor máximo (controlado por SHOW_VALUE_LABELS)
        if SHOW_VALUE_LABELS and max_value > 0:
            if log_scale and pivoted.values.min() > 0:
                label_y_pos = max_value * 1.05
            else:
                label_y_pos = max_value * 1.01
                
            ax.text(
                bar.get_x() + bar.get_width()/2,
                label_y_pos,
                f"{max_value:.4g}",
                ha='center',
                va='bottom',
                fontweight='bold',
                color='black',
                fontsize=11
            )
    
    # Leyenda a la derecha
    legend_title = {
        'framework': 'Framework',
        'machine': 'Machine',
        'machine_en': 'Machine',
        'framework_machine': 'Framework (Machine)'
    }.get(group_col, group_col.replace('_', ' ').title())
    
    legend = plt.legend(title=legend_title, 
                       frameon=True, loc='center left', bbox_to_anchor=(1.0, 0.5))
    legend.get_frame().set_facecolor('#F8F8F8')
    legend.get_frame().set_edgecolor('#CCCCCC')
    
    # Grid y layout
    plt.grid(axis='both', linestyle='--', alpha=0.7)
    plt.tight_layout(rect=[0, 0, 0.85, 1])
    #
    plt.show()

# 5. Función para mostrar líneas de tendencia con R²
# ---------------------------------------------------
def plot_trend_lines(data, x_col, group_col, y_col, title, ylabel, log_scale=False):
    """
    Creates a line chart with polynomial trend lines and R² values in the legend
    
    Args:
        data: DataFrame with the data
        x_col: Column for X axis (usually 'config')
        group_col: Column to group by (different lines)
        y_col: Column with values to plot
        title: Chart title
        ylabel: Y axis label
        log_scale: Whether to use log scale for Y axis
    """
    # Creamos el gráfico
    fig, ax = plt.subplots(figsize=(14, 7))
    
    # Agrupamos los datos
    groups = data[group_col].unique()
    
    # Para configuraciones, queremos un orden específico
    if x_col == 'config':
        x_values = config_order
        # Creamos un mapa del índice de cada configuración
        x_indices = {config: i for i, config in enumerate(config_order)}
    else:
        x_values = sorted(data[x_col].unique())
        x_indices = {x: i for i, x in enumerate(x_values)}
    
    # Almacenar los valores R² para incluirlos en la leyenda
    r_squared_values = {}
    
    # Para cada grupo, dibujamos una línea
    for group in groups:
        # Filtramos los datos del grupo
        group_data = data[data[group_col] == group]
        
        # Preparamos los datos para la gráfica
        group_x = []
        group_y = []
        
        # Ordenamos por x_col si no es config (que ya tiene un orden definido)
        if x_col != 'config':
            group_data = group_data.sort_values(by=x_col)
        
        # Para cada valor x, obtenemos el valor y correspondiente
        for x_val in x_values:
            x_data = group_data[group_data[x_col] == x_val]
            if len(x_data) > 0:
                # Si hay datos para este valor x, añadimos el punto
                y_val = x_data[y_col].values[0]
                # Para escala logarítmica, aseguramos que los valores sean positivos
                if log_scale and (y_val <= 0 or np.isnan(y_val)):
                    continue
                group_x.append(x_indices[x_val])
                group_y.append(y_val)
        
        # Definimos el color según el grupo
        if group_col == 'framework_machine':
            color = framework_machine_colors.get(group, "#333333")
        elif group_col == 'framework':
            color = framework_colors.get(group, "#333333")
        elif group_col in ['machine', 'machine_en']:
            color = machine_colors.get(group, "#333333")
        else:
            # Color por defecto
            color = None
        
        # Si hay suficientes datos para dibujar y ajuste polinómico
        if len(group_x) >= 3:
            # Dibujamos los puntos y la línea
            line, = ax.plot(group_x, group_y, 'o-', 
                          label=group,
                          color=color,
                          linewidth=2, markersize=8)
            
            # Añadimos etiquetas de valores solo si SHOW_VALUE_LABELS es True
            if SHOW_VALUE_LABELS:
                for j, val in enumerate(group_y):
                    if val > 0 or not log_scale:
                        ax.text(group_x[j], val * (1.05 if log_scale else 1.01), 
                               f"{val:.4g}", ha='center', va='bottom',
                               color=color,
                               fontsize=9, fontweight='bold')
            
            # Usamos un ajuste polinómico (grado 2) para capturar cambios no lineales
            z = np.polyfit(group_x, group_y, 2)
            p = np.poly1d(z)
            
            # Generamos puntos adicionales para suavizar la curva
            x_smooth = np.linspace(min(group_x), max(group_x), 100)
            y_smooth = p(x_smooth)
            
            # Dibujamos la línea de tendencia suavizada
            ax.plot(x_smooth, y_smooth, '--', 
                   color=color, 
                   linewidth=1.5, alpha=0.7)
            
            # Calculamos R² para evaluar la calidad del ajuste
            y_pred = p(group_x)
            ss_total = np.sum((group_y - np.mean(group_y))**2)
            ss_residual = np.sum((group_y - y_pred)**2)
            r_squared = 1 - (ss_residual / ss_total)
            r_squared_values[group] = r_squared
            
        # Si hay datos pero no suficientes para ajuste polinómico (2 o menos puntos)
        elif len(group_x) > 0:
            # Dibujamos solo los puntos y la línea
            line, = ax.plot(group_x, group_y, 'o-', 
                          label=group,
                          color=color,
                          linewidth=2, markersize=8)
            
            # Añadimos etiquetas si está habilitado
            if SHOW_VALUE_LABELS:
                for j, val in enumerate(group_y):
                    if val > 0 or not log_scale:
                        ax.text(group_x[j], val * (1.05 if log_scale else 1.01), 
                               f"{val:.4g}", ha='center', va='bottom',
                               color=color,
                               fontsize=9, fontweight='bold')
            
            # Para linea recta necesitamos al menos 2 puntos
            if len(group_x) >= 2:
                # Calculamos la regresión lineal simple como respaldo
                slope, intercept, r_value, p_value, std_err = stats.linregress(group_x, group_y)
                
                # Dibujamos la línea de tendencia lineal
                trend_x = [min(group_x), max(group_x)]
                trend_y = [slope * x + intercept for x in trend_x]
                ax.plot(trend_x, trend_y, '--', color=color, linewidth=1.5, alpha=0.7)
                
                # Guardamos el valor R² para la leyenda
                r_squared = r_value ** 2
                r_squared_values[group] = r_squared
            else:
                r_squared_values[group] = float('nan')
    
    # Configuración de escala y etiquetas
    if log_scale:
        ax.set_yscale('log')
    
    ax.set_title(f"{title}", fontweight='bold', pad=20)
    ax.set_xlabel("Configuration" if x_col == 'config' else x_col.replace('_', ' ').title(), fontweight='bold')
    ax.set_ylabel(ylabel, fontweight='bold')
    
    # Si es configuración, usamos las etiquetas específicas
    if x_col == 'config':
        ax.set_xticks(range(len(x_values)))
        ax.set_xticklabels(x_values, rotation=30, ha='right')
    
    # Añadir áreas sombreadas para separar tamaños de población si estamos usando configuraciones
    if x_col == 'config':
        pop_sizes = ["2^6", "2^10", "2^14"]
        colors = ["#f0f0f0", "#e8e8e8", "#f0f0f0"]
        
        for p_idx, pop in enumerate(pop_sizes):
            pop_configs = [j for j, c in enumerate(config_order) if c.startswith(pop)]
            if pop_configs:
                start, end = min(pop_configs), max(pop_configs)
                ax.axvspan(start - 0.5, end + 0.5, alpha=0.2, color=colors[p_idx])
                ax.text((start + end) / 2, 0.98, 
                       f"Population {pop}", ha='center', fontsize=10, alpha=0.7,
                       transform=ax.get_xaxis_transform())
    
    # Crear etiquetas personalizadas para la leyenda que incluyan R²
    custom_labels = []
    custom_handles = []
    
    # Obtenemos los handles actuales y sus etiquetas
    handles, labels = ax.get_legend_handles_labels()
    
    for handle, label in zip(handles, labels):
        if label in r_squared_values:
            r2 = r_squared_values[label]
            # Incluimos el valor R² en la etiqueta SIEMPRE
            custom_labels.append(f"{label} (R²={r2:.2f})")
        else:
            custom_labels.append(label)
        custom_handles.append(handle)
    
    # Creamos la leyenda con las etiquetas personalizadas
    legend_title = {
        'framework': 'Framework',
        'machine': 'Machine',
        'machine_en': 'Machine',
        'framework_machine': 'Framework (Machine)'
    }.get(group_col, group_col.replace('_', ' ').title())
    
    ax.legend(custom_handles, custom_labels, title=legend_title, 
              frameon=True, loc='center left', bbox_to_anchor=(1.0, 0.5))
    
    # Grid y layout
    ax.grid(axis='both', linestyle='--', alpha=0.7)
    plt.tight_layout(rect=[0, 0, 0.88, 1])
    #
    plt.show()
# 6. Visualización de resultados: Fitness Máximo por Consumo
# ----------------------------------------------------------

# Primero: Gráfico para cada máquina por separado - CORREGIDO
for machine, machine_en in zip(['ordenador', 'pablo'], ['Laptop', 'Server']):
    machine_data = df_combined[df_combined['machine'] == machine]
    
    plot_grouped_bars(
        data=machine_data,
        x_col="config",
        group_col="framework",
        y_col="fitness_per_energy",
        title=f"Energy Efficiency: Fitness/Energy ({machine_en})",
        ylabel="Fitness per kWh",
        log_scale=True
    )
    
    # También graficamos con líneas de tendencia
    plot_trend_lines(
        data=machine_data,
        x_col="config",
        group_col="framework",
        y_col="fitness_per_energy",
        title=f"Energy Efficiency: Fitness/Energy ({machine_en}) with Trend Lines",
        ylabel="Fitness per kWh",
        log_scale=True
    )

# Segundo: Separamos por framework y comparamos máquinas
for framework in df_combined['framework'].unique():
    fw_data = df_combined[df_combined['framework'] == framework]
    
    plot_grouped_bars(
        data=fw_data,
        x_col="config",
        group_col="machine_en",
        y_col="fitness_per_energy",
        title=f"Energy Efficiency: Fitness/Energy ({framework})",
        ylabel="Fitness per kWh",
        log_scale=True
    )
    
    # También graficamos con líneas de tendencia
    plot_trend_lines(
        data=fw_data,
        x_col="config",
        group_col="machine_en",
        y_col="fitness_per_energy",
        title=f"Energy Efficiency: Fitness/Energy ({framework}) with Trend Lines",
        ylabel="Fitness per kWh",
        log_scale=True
    )

# Tercero: Todo junto - cada barra es una combinación framework-máquina
plot_grouped_bars(
    data=df_combined,
    x_col="config",
    group_col="framework_machine",
    y_col="fitness_per_energy",
    title="Energy Efficiency: Fitness/Energy (All Combinations)",
    ylabel="Fitness per kWh",
    log_scale=True
)

# Graficamos con líneas de tendencia
plot_trend_lines(
    data=df_combined,
    x_col="config",
    group_col="framework_machine",
    y_col="fitness_per_energy",
    title="Energy Efficiency: Fitness/Energy (All Combinations) with Trend Lines",
    ylabel="Fitness per kWh",
    log_scale=True
)

# 7. Visualización de resultados: Variación de Fitness por Consumo
# ----------------------------------------------------------------

# Primero: Gráfico para cada máquina por separado - CORREGIDO
for machine, machine_en in zip(['ordenador', 'pablo'], ['Laptop', 'Server']):
    machine_data = df_combined[df_combined['machine'] == machine]
    
    plot_grouped_bars(
        data=machine_data,
        x_col="config",
        group_col="framework",
        y_col="variacion_fitness_per_energy",
        title=f"Energy Efficiency: Fitness Variation/Energy ({machine_en})",
        ylabel="Fitness Variation per kWh",
        log_scale=True
    )
    
    # También graficamos con líneas de tendencia
    plot_trend_lines(
        data=machine_data,
        x_col="config",
        group_col="framework",
        y_col="variacion_fitness_per_energy",
        title=f"Energy Efficiency: Fitness Variation/Energy ({machine_en}) with Trend Lines",
        ylabel="Fitness Variation per kWh",
        log_scale=True
    )

# Segundo: Separamos por framework y comparamos máquinas
for framework in df_combined['framework'].unique():
    fw_data = df_combined[df_combined['framework'] == framework]
    
    plot_grouped_bars(
        data=fw_data,
        x_col="config",
        group_col="machine_en",
        y_col="variacion_fitness_per_energy",
        title=f"Energy Efficiency: Fitness Variation/Energy ({framework})",
        ylabel="Fitness Variation per kWh",
        log_scale=True
    )
    
    # También graficamos con líneas de tendencia
    plot_trend_lines(
        data=fw_data,
        x_col="config",
        group_col="machine_en",
        y_col="variacion_fitness_per_energy",
        title=f"Energy Efficiency: Fitness Variation/Energy ({framework}) with Trend Lines",
        ylabel="Fitness Variation per kWh",
        log_scale=True
    )

# Tercero: Todo junto - cada barra es una combinación framework-máquina
plot_grouped_bars(
    data=df_combined,
    x_col="config",
    group_col="framework_machine",
    y_col="variacion_fitness_per_energy",
    title="Energy Efficiency: Fitness Variation/Energy (All Combinations)",
    ylabel="Fitness Variation per kWh",
    log_scale=True
)

# Graficamos con líneas de tendencia
plot_trend_lines(
    data=df_combined,
    x_col="config",
    group_col="framework_machine",
    y_col="variacion_fitness_per_energy",
    title="Energy Efficiency: Fitness Variation/Energy (All Combinations) with Trend Lines",
    ylabel="Fitness Variation per kWh",
    log_scale=True
)

# 8. Comparativas adicionales de distribuciones
# --------------------------------------------
plt.figure(figsize=(12, 6))
sns.boxplot(x='framework', y='fitness_per_energy', hue='machine_en', data=df_combined)
plt.title('Energy Efficiency Distribution by Framework and Machine', fontweight='bold')
plt.xlabel('Framework', fontweight='bold')
plt.ylabel('Fitness per kWh', fontweight='bold')
plt.yscale('log')  # Escala logarítmica si hay grandes diferencias
plt.grid(axis='both', linestyle='--', alpha=0.7)
plt.legend(title='Machine')
plt.tight_layout()

plt.show()

plt.figure(figsize=(12, 6))
sns.boxplot(x='framework', y='variacion_fitness_per_energy', hue='machine_en', data=df_combined)
plt.title('Fitness Variation Efficiency Distribution by Framework and Machine', fontweight='bold')
plt.xlabel('Framework', fontweight='bold')
plt.ylabel('Fitness Variation per kWh', fontweight='bold')
plt.yscale('log')  # Escala logarítmica si hay grandes diferencias
plt.grid(axis='both', linestyle='--', alpha=0.7)
plt.legend(title='Machine')
plt.tight_layout()

plt.show()